<a href="https://colab.research.google.com/github/artkula/ML-retreat-tekmek-2025/blob/main/NN_RNN_SINDy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks, Recurrent Neural Networks, and SINDy for Mechanical Systems

## Learning Objectives

By completing this notebook, you will be able to:

- **Apply neural networks** to learn constitutive material behavior from experimental data
- **Understand when history matters** and select appropriate architectures (feedforward vs recurrent)
- **Implement sparse discovery** to extract interpretable governing equations from data
- **Validate models systematically** using interpolation, extrapolation, and physical constraints
- **Make informed engineering decisions** about method selection based on problem characteristics

## Notebook Organization

This notebook progressively builds your understanding through four carefully designed levels:

1. **Level 1: Hyperelastic material** (no history, clean data) → Master feedforward networks
2. **Level 2: Noisy hyperelastic data** → Learn regularization and overfitting control
3. **Level 3: Viscoelastic material** (history-dependent) → Discover why RNNs are necessary
4. **Level 4: Sparse discovery with SINDy** → Extract interpretable equations from data

Each section includes:
- **Conceptual foundation** before any code
- **Prediction exercises** to test your intuition
- **Detailed code explanations** connecting choices to physics
- **Critical analysis** of results and limitations

---

## Why These Methods Matter in Mechanics

**Real-world challenge**: Material characterization requires expensive experimental campaigns. A single stress-strain curve might involve specialized equipment, careful sample preparation, and hours of testing. For complex materials (hyperelastic rubbers, viscoelastic polymers, composites), traditional approaches require testing at multiple rates, temperatures, and loading paths: potentially hundreds of experiments costing months of work.

**Traditional limitations**:
- Constitutive models are hand-crafted with parameters fitted to specific datasets
- Extending to new loading conditions requires new experiments and model adjustments
- History-dependent materials (viscoelasticity, plasticity) require differential equations that are difficult to calibrate
- Interpolation between test conditions is often unreliable

**Key insight**: Machine learning can learn material behavior from limited data, but only when combined with proper domain knowledge. Neural networks provide flexible function approximation, recurrent architectures handle history dependence, and sparse identification (SINDy) discovers interpretable governing equations.

**Engineering context**: This matters for:
- **Cost reduction**: Cheaper experimental campaign with ML-guided testing
- **Digital twins**: Real-time simulation for design optimization
- **Predictive maintenance**: Material degradation monitoring
- **New material discovery**: Rapid screening of material candidates

---

## Physical Background: Stress and Strain Measures

### Fundamental Relationships

We work with two stress measures that are fundamental in finite deformation mechanics:

**Second Piola-Kirchhoff stress** $S$ $[\text{kPa}]$:
- Work-conjugate to Green-Lagrange strain
- Defined in reference (undeformed) configuration
- Symmetric tensor, computationally convenient

**Cauchy stress** $\sigma$ $[\text{kPa}]$:
- True stress acting on deformed configuration
- What you actually measure in experiments
- Physical interpretation: force per unit deformed area

For incompressible materials under uniaxial loading with stretch $\lambda$ $[-]$:

$$\sigma = \lambda^2 S$$

This relationship is exact for incompressible materials and will be our validation check throughout.

**Stretch ratio** $\lambda = L/L_0$:
- $\lambda = 1$: No deformation (reference state)
- $\lambda > 1$: Extension (stretching)
- $\lambda < 1$: Compression (shortening)

---

### Why use stretch $\lambda$ instead of strain $\varepsilon$?

**Physical reasoning:**
- **Strain** ($\varepsilon = \Delta L/L_0 = \lambda - 1$) is additive but fails at large deformations
- **Stretch** ($\lambda = L/L_0$) is multiplicative and physically meaningful for finite deformations

**Mathematical advantages:**
- Strain energy $W(\lambda)$ remains well-defined for all deformations
- Incompressibility constraint: $\det(\mathbf{F}) = \lambda_1 \lambda_2 \lambda_3 = 1$ (simple in stretch)
- Stress-stretch relations avoid singularities at large deformations

**Example:** For 100% extension:
- Strain: $\varepsilon = 1.0$ (additive)
- Stretch: $\lambda = 2.0$ (multiplicative)
  
If you stretch twice: $\varepsilon_1 + \varepsilon_2 \neq \varepsilon_{\text{total}}$, but $\lambda_1 \times \lambda_2 = \lambda_{\text{total}}$ ✓

For hyperelastic materials undergoing large deformations ($\lambda > 1.5$), stretch provides the correct mathematical framework. Small strain theory ($\varepsilon \approx \lambda - 1$) is only valid for $\lambda < 1.1$ (< 10% deformation).

---

### Why $\sigma = \lambda^2 S$?

**Physical interpretation:**

The factor $\lambda^2$ arises from the transformation between reference and current configurations:

1. **$S$ (Second Piola-Kirchhoff)**: Stress measured in the **undeformed** (reference) state
   - Force per unit original area: $F_0/A_0$

2. **$\sigma$ (Cauchy)**: Stress measured in the **deformed** (current) state  
   - Force per unit current area: $F/A$

3. **For incompressible uniaxial tension:**
   - Cross-sectional area shrinks: $A = A_0/\lambda$ (volume preservation)
   - The push-forward tensor operation: $\boldsymbol{\sigma} = \mathbf{F} \cdot \mathbf{S} \cdot \mathbf{F}^T / \det(\mathbf{F})$
   - For incompressible ($\det(\mathbf{F}) = 1$) and uniaxial $\mathbf{F} = \text{diag}(\lambda, \lambda^{-1/2}, \lambda^{-1/2})$:
   
   $$\sigma = \lambda \cdot S \cdot \lambda = \lambda^2 S$$

**In words:** Cauchy stress accounts for both the geometric area change AND the stress measure transformation from reference to current configuration.

---

## Critical Warning About Extrapolation

⚠️ **Neural Networks Cannot Extrapolate Reliably** ⚠️

The feedforward neural network is trained on data where $\lambda \in [1.0, \lambda_{max}]$.

**Extrapolation to $\lambda < 1.0$ (compression) produces unphysical results** because:
1. The network has never seen compression data
2. Neural networks learn correlations, not physical laws
3. Outside the training domain, predictions are essentially random

For reliable predictions outside the training range, consider:
- **Including boundary conditions**: Add anchor point at $\lambda = 1$ with $\sigma = 0$
- **Physics-informed loss**: Penalty term enforcing $\sigma(1) \approx 0$
- **PINNs**: Physics-informed neural networks with PDE constraints
- **Hybrid models**: Combine NN with known physical structure

Remember: A 99% accurate model inside training domain may be 0% accurate outside!

## Setup: Environment Configuration and Reproducibility

This section configures the computational environment with careful attention to:
- **Reproducibility**: Fixed random seeds across numpy, Python, and Keras
- **Visualization**: Consistent, publication-quality plotting defaults
- **Performance**: FAST mode for CPU execution during development
- **Compatibility**: Graceful handling of Keras 3 vs TensorFlow Keras

### What to Expect
When you run this cell, you should see:
- Confirmation of FAST mode status (ON for quick testing, OFF for full results)
- Random seed value (default 1827, can be overridden via PY_SEED environment variable)
- Keras version being used (prefer standalone Keras 3 for latest features)
- Successful loading of helper functions

In [ ]:
# ============================================================================
# GLOBAL SETUP: IMPORTS, SEEDS, AND PLOTTING DEFAULTS
# ============================================================================
# This cell must run first and sets up the entire computational environment.
# Every choice here affects reproducibility and visual quality of results.

import os, random, warnings
warnings.filterwarnings("ignore")  # Suppress TensorFlow/Keras warnings for cleaner output

# ============================================================================
# EXECUTION MODES AND RANDOM SEEDS
# ============================================================================
# FAST mode: Reduces dataset sizes and training epochs for CPU execution
#   - ON (default): Quick testing, ~2 minutes total runtime
#   - OFF: Full training, ~10 minutes on GPU, ~30 minutes on CPU
FAST = bool(int(os.environ.get("FAST", "1")))

# SEED: Master random seed for reproducibility
#   - Controls: numpy, random, and keras random number generation
#   - Default: 1827 (arbitrary but fixed for course consistency)
#   - Override: Set PY_SEED environment variable
SEED = int(os.environ.get("PY_SEED", "1827"))

# Apply seeds immediately for reproducibility
import numpy as np
np.random.seed(SEED)
random.seed(SEED)

# ============================================================================
# DEEP LEARNING FRAMEWORK SELECTION
# ============================================================================
# Prefer standalone Keras 3 (better features) but fall back to tf.keras if needed
try:
    import keras  # Standalone Keras 3 (recommended)
    print("✅ Using standalone Keras 3 (recommended)")
except Exception:
    from tensorflow import keras  # TensorFlow's built-in Keras
    print("⚠️ Using TensorFlow's built-in Keras (older version)")

from keras import layers, callbacks, regularizers

# ============================================================================
# SCIENTIFIC COMPUTING AND VISUALIZATION
# ============================================================================
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp  # For ODE integration (Lorenz system)
from tqdm.auto import tqdm  # Progress bars for long operations

# ============================================================================
# PYSINDY INSTALLATION AND IMPORT
# ============================================================================
# PySINDy (Sparse Identification of Nonlinear Dynamics) for discovering equations
try:
    import pysindy as ps
except Exception:
    print("📦 Installing PySINDy (required for sparse discovery)...")
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pysindy", "--quiet"])
    import pysindy as ps
    print("✅ PySINDy installed successfully")

# Set Keras seed (must be after keras import)
keras.utils.set_random_seed(SEED)

# ============================================================================
# MATPLOTLIB CONFIGURATION FOR PUBLICATION-QUALITY PLOTS
# ============================================================================
# These settings ensure consistent, professional-looking figures throughout
mpl.rcParams.update({
    # Figure size and resolution
    "figure.figsize": (7, 4.5),        # Default size in inches (16:10 aspect ratio)
    "figure.dpi": 120,                 # Screen resolution (points per inch)
    "savefig.dpi": 200,                # Saved figure resolution (higher for papers)

    # Background colors
    "figure.facecolor": "white",       # White figure background
    "savefig.facecolor": "white",      # White background when saving

    # Grid appearance
    "axes.grid": True,                 # Show grid by default
    "grid.alpha": 0.2,                 # Subtle grid (20% opacity)

    # Font sizes (scaled for readability)
    "font.size": 12,                   # Base font size
    "axes.titlesize": 13,              # Slightly larger titles
    "axes.labelsize": 12,              # Axis label size
    "legend.fontsize": 11,             # Legend text
    "xtick.labelsize": 11,             # X-axis tick labels
    "ytick.labelsize": 11,             # Y-axis tick labels

    # Layout
    "figure.constrained_layout.use": False  # Use tight_layout instead (more reliable)
})

# Color palette: ColorBrewer Set1 (colorblind-friendly, distinct colors)
# Blue, Orange, Green, Purple, Brown, Pink - optimized for clarity
mpl.rcParams["axes.prop_cycle"] = mpl.cycler(
    color=["#1f77b4", "#ff7f0e", "#2ca02c", "#9467bd", "#8c564b", "#e377c2"]
)

print(f"🚀 Environment configured:")
print(f"   FAST mode: {'⚡ ON (reduced data/epochs)' if FAST else '🐢 OFF (full training)'}")
print(f"   Random seed: {SEED}")
print(f"   Keras version: {keras.__version__}")

In [ ]:
# ============================================================================
# HELPER FUNCTIONS FOR CONSISTENT PLOTTING AND ANALYSIS
# ============================================================================
# These utilities ensure uniform appearance and correct physical units throughout

def set_labels(ax, x_label=None, y_label=None, title=None):
    """Apply labels to axes with consistent formatting."""
    if x_label: ax.set_xlabel(x_label)
    if y_label: ax.set_ylabel(y_label)
    if title:   ax.set_title(title)

def label_units(var: str, unit: str) -> str:
    """Format variable with units using LaTeX: 'σ [kPa]'."""
    return rf"{var} [{unit}]"

def dimless(var: str) -> str:
    """Format dimensionless variable: 'λ [–]'."""
    return rf"{var} [–]"

def legend_outside_right(ax, loc="center left", anchor=(1.02, 0.5)):
    """Place legend outside plot area to avoid obscuring data."""
    ax.legend(loc=loc, bbox_to_anchor=anchor, borderaxespad=0.0)

def set_equal_aspect_2d(ax):
    """Force square aspect ratio (important for phase plots)."""
    try:
        ax.set_aspect("equal", adjustable="box")
    except Exception:
        pass  # Some plot types don't support equal aspect

def phase_axes(ax, xlim=None, ylim=None):
    """Configure axes for phase space plots with equal scaling."""
    if xlim: ax.set_xlim(*xlim)
    if ylim: ax.set_ylim(*ylim)
    set_equal_aspect_2d(ax)

def plot_residuals(t, resid, ax=None, title="Residuals", units=None,
                   symmetric=True, annotate_stats=True):
    """
    Plot model residuals with statistical annotations.

    Parameters:
        t: Time array
        resid: Residual array (prediction - truth)
        ax: Matplotlib axes (creates new if None)
        title: Plot title
        units: Physical units for y-axis label
        symmetric: If True, center y-axis on zero with symmetric limits
        annotate_stats: If True, show RMSE in text box
    """
    ax = ax or plt.gca()
    ax.plot(t, resid, lw=1.5)
    ax.axhline(0.0, color="k", lw=1.0, alpha=0.6)  # Zero reference line

    if symmetric:
        # Symmetric limits help visualize bias vs variance
        L = np.nanmax(np.abs(resid))
        if np.isfinite(L) and L > 0:
            ax.set_ylim(-1.05 * L, 1.05 * L)

    yl = "Error" if units is None else f"Error [{units}]"
    set_labels(ax, x_label=label_units(r"$t$", "s"), y_label=yl, title=title)

    if annotate_stats:
        rmse = float(np.sqrt(np.nanmean(resid**2)))
        ax.text(0.01, 0.95, f"RMSE: {rmse:.3g}", transform=ax.transAxes,
                va="top", ha="left",
                bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))
    return ax

# ============================================================================
# 3D PLOTTING FOR DYNAMICAL SYSTEMS
# ============================================================================
from mpl_toolkits.mplot3d import Axes3D

def plot_trajectory_3d(x, y, z, elev=25, azim=35, lw=1.5, title="3D Trajectory"):
    """
    Create 3D trajectory plot with fixed aspect ratio and viewing angle.

    Parameters:
        x, y, z: Coordinate arrays
        elev: Elevation viewing angle in degrees
        azim: Azimuthal viewing angle in degrees
        lw: Line width
        title: Plot title

    Returns:
        fig, ax: Matplotlib figure and 3D axes
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.plot(x, y, z, lw=lw)
    ax.set_title(title)

    # Equal aspect ratio in 3D (tricky but important for accurate visualization)
    try:
        ranges = [np.max(v) - np.min(v) for v in [x, y, z]]
        max_range = max(ranges) if max(ranges) > 0 else 1.0

        # Find center point
        cx = 0.5 * (np.max(x) + np.min(x))
        cy = 0.5 * (np.max(y) + np.min(y))
        cz = 0.5 * (np.max(z) + np.min(z))

        # Set equal ranges centered on data
        ax.set_xlim(cx - 0.5*max_range, cx + 0.5*max_range)
        ax.set_ylim(cy - 0.5*max_range, cy + 0.5*max_range)
        ax.set_zlim(cz - 0.5*max_range, cz + 0.5*max_range)
        ax.set_box_aspect((1, 1, 1))
    except Exception:
        pass  # Fallback for older matplotlib versions

    ax.view_init(elev=elev, azim=azim)
    return fig, ax

print("✅ Helper functions loaded successfully")

## Level 1: Hyperelastic Material (Clean Data, No History)

### Why This Matters

**Engineering scenario**: You have expensive experimental data from material testing and need to predict behavior at intermediate conditions. A hyperelastic material (like rubber) has no memory - its stress depends only on current stretch, not loading history.

**Learning goals**:
1. Understand when feedforward networks are appropriate (no history dependence)
2. Learn proper data normalization for neural networks
3. Recognize extrapolation failures and their implications
4. Master the train/validation/test split paradigm

### Conceptual Foundation

**Hyperelastic constitutive model** (Mooney-Rivlin):

For incompressible materials, the strain energy density is:
$$W = C_1(I_1 - 3) + C_2(I_2 - 3)$$

where $I_1 = \lambda_1^2 + \lambda_2^2+\lambda_3^2, I_2 = \lambda_1^2\lambda_2^2+\lambda_1^2\lambda_3^2+\lambda_2^2\lambda_3^2$ are strain invariants and $C_1, C_2$ are material parameters. The second Piola-Kirchoff stress is given by $S_k = \frac{\partial W}{\partial \lambda_k} + p$, where $p$ is the pressure determined from the boundary conditions.

In 1D case, we have $\lambda_1 = \lambda, \lambda_2=\lambda_3 = 1/\sqrt{\lambda}$ and $S_2=S_3=0$. This follows from the incompressibility constraint $\det(F) = \lambda_1 \lambda_2 \lambda_3 = 1$ (volume preservation) combined with the symmetry assumption $\lambda_2 = \lambda_3$ (equal transverse contraction). Substituting $\lambda_1 = \lambda$ gives $\lambda \times \lambda_2^2 = 1$, which yields $\lambda_2 = \lambda_3 = 1/\sqrt{\lambda}$. The the pressure $p$ is determined by the boundary condition $S_2=S_3=0$ (no stress on sides):

$$S_2 = \frac{\partial W}{\partial \lambda_2} + p = 0$$

This gives:

$$p = -\frac{\partial W}{\partial \lambda_2}$$

This leads to the stress-stretch relationship we will learn:
$$S_1 = \frac{1}{\lambda^2}\left[2C_1(\lambda^2 - \frac{1}{\lambda}) + 2C_2(\lambda - \frac{1}{\lambda^2})\right]$$

**Key properties**:
- Path-independent: Same stretch → same stress (no hysteresis)
- Nonlinear: Stiffens significantly at large stretches
- Physical constraint: $S(\lambda=1) = 0$ (zero stress at zero strain)

### Before We Code: Prediction Exercise

Consider the stress-stretch curve we are about to generate:
- What shape do you expect? (Linear? Exponential? Polynomial?)
- Will our training data capture the full nonlinearity?
- What physical constraint must the model satisfy at $\lambda = 1$?
- Why might extrapolation to compression ($\lambda < 1$) fail?

**Answer hints**:
- The curve is nonlinear, stiffening at large stretches
- Training only covers extension ($\lambda > 1$), missing compression behavior
- At $\lambda = 1$, stress must be exactly zero (reference state)
- Without compression data, the network cannot learn the symmetric behavior

In [ ]:
# ============================================================================
# DATA GENERATION: MOONEY-RIVLIN HYPERELASTIC MODEL
# ============================================================================
# We generate synthetic but physically realistic data based on a well-established
# constitutive model used for rubber-like materials.

# Material parameters (typical for soft rubber, experimentally determined)
C1 = 31.68  # First Mooney-Rivlin parameter [kPa]
C2 = 34.70  # Second Mooney-Rivlin parameter [kPa]
# These values give realistic stress magnitudes for biological tissues or soft elastomers

def compute_S_stress(stretch):
    '''
    Compute Second Piola-Kirchhoff stress from Mooney-Rivlin model.

    This is the GROUND TRUTH that our neural network will try to learn.
    Notice the highly nonlinear relationship - this is what makes the problem
    interesting and challenging for simple interpolation methods.

    Parameters:
        stretch: Stretch ratio λ (1.0 = undeformed, >1 = extension)

    Returns:
        S: Second Piola-Kirchhoff stress [kPa]
    '''
    lam = stretch  # More readable notation

    # Mooney-Rivlin model for incompressible material
    # Note the 1/λ terms - these blow up as λ→0, making compression very stiff
    S = (2*C1*(lam**2 - 1/lam) + 2*C2*(lam - 1/lam**2)) / lam**2

    return S

# ============================================================================
# TRAINING DATA GENERATION
# ============================================================================
# KEY DECISION: How many data points do we need?
#   - Too few: Cannot capture nonlinearity (underfitting)
#   - Too many: Expensive experiments in real applications
#   - Sweet spot: 10-15 points for this level of nonlinearity

N_points = 10 if FAST else 15  # Reduce for CPU testing, full for GPU

# Stretch range: 1.0 (undeformed) to 5.8 (480% extension)
# This is realistic for rubber - beyond this, material might fail
stretch_train = np.linspace(1.0, 5.8, N_points)

# Compute corresponding stresses (our "experimental measurements")
stress_train = compute_S_stress(stretch_train)

# ============================================================================
# VISUALIZATION: UNDERSTANDING OUR TRAINING DATA
# ============================================================================
plt.figure(figsize=(8, 5))

# Scatter plot emphasizes discrete nature of experimental data
plt.scatter(stretch_train, stress_train,
           s=100,                    # Large markers for visibility
           alpha=0.7,                # Slight transparency
           edgecolors='black',       # Black edges for clarity
           linewidth=2,              # Thick edges
           label=f'Training data (n={N_points})',
           zorder=5)                 # Draw on top

# Add smooth curve to show true function (what we're trying to learn)
stretch_smooth = np.linspace(1.0, 5.8, 200)
stress_smooth = compute_S_stress(stretch_smooth)
plt.plot(stretch_smooth, stress_smooth,
        'b-', alpha=0.3, linewidth=1,
        label='True function', zorder=1)

plt.xlabel('Stretch λ [-]')
plt.ylabel('Second Piola-Kirchhoff Stress S [kPa]')
plt.title('Hyperelastic Training Data: What the Neural Network Sees')
plt.legend()
plt.grid(True, alpha=0.3)

# Add annotations for key insights
plt.annotate('Zero stress at λ=1\n(undeformed state)',
            xy=(1, 0), xytext=(1.5, 20),
            arrowprops=dict(arrowstyle='->', alpha=0.5))
plt.annotate('Nonlinear stiffening\nat large stretches',
            xy=(5, stress_train[-2]), xytext=(3.5, 150),
            arrowprops=dict(arrowstyle='->', alpha=0.5))

plt.tight_layout()
plt.show()

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================
print(f"📊 Training Data Summary:")
print(f"   Number of points: {N_points}")
print(f"   Stretch range: λ ∈ [{stretch_train.min():.2f}, {stretch_train.max():.2f}]")
print(f"   Stress range: S ∈ [{stress_train.min():.1f}, {stress_train.max():.1f}] kPa")
print(f"   Nonlinearity measure: {stress_train[-1]/stress_train[1]:.1f}x stress increase")
print(f"\n⚠️ Critical limitation: No compression data (λ < 1)")

### Neural network architecture and training

Now we build and train a feedforward neural network to learn the stress-stretch relationship.

#### Architecture design decisions

**Network depth and width:**

- **Shallow (2-3 layers):** Fast training and inference, but may underfit nonlinear relationships due to limited representational capacity
- **Deep (6-8 layers):** Can capture complex nonlinear patterns and hierarchical features, but carries risk of overfitting with limited data
- **Wide (64-128 neurons):** Provides greater capacity per layer, better suited for approximating smooth functions
- **Narrow (16-32 neurons):** Forces feature compression, acts as implicit regularization to prevent overfitting

**Activation functions:**

- **Tanh:** Smooth and bounded output in [-1,1], well-suited for normalized data, potential gradient saturation in deep networks
- **ReLU:** Computationally efficient, unbounded positive output, susceptible to dead neuron problem, creates piecewise linear approximations
- **Swish/GELU:** Modern alternatives with smooth non-saturating behavior, slightly higher computational cost

**Our choice:** 6 layers with 32 neurons per layer using tanh activation (provides smooth, physics-consistent approximations)

---

#### Loss functions for training

When training a neural network, we require a **loss function** (objective function) that quantifies the discrepancy between predictions and true values. The optimizer minimizes this loss during training.

**Mean squared error (MSE):**

$$L = \frac{1}{n}\sum_{i=1}^{n}(\hat{y}_i - y_i)^2$$

- Penalizes large errors quadratically through the squaring operation
- Smooth and differentiable everywhere, facilitating stable gradient-based optimization
- Standard choice for regression problems with continuous outputs
- Well-suited for stress-strain prediction where smooth gradients are desired

**Alternative loss functions:**

- **Mean absolute error (MAE):** $L = \frac{1}{n}\sum_{i=1}^{n}|\hat{y}_i - y_i|$ — more robust to outliers, but exhibits gradient discontinuity at zero
- **Huber loss:** Combines MSE for small errors and MAE for large errors — balances smoothness with robustness to outliers

For our stress-strain prediction problem, MSE is the appropriate choice because: (1) the data is relatively clean without significant outliers, (2) smooth gradients are essential for stable training convergence, and (3) quadratic penalization naturally emphasizes reduction of larger errors, which is desirable for capturing the full range of stress-strain behavior.

In [ ]:
# ============================================================================
# NEURAL NETWORK: TRAINING ON HYPERELASTIC DATA (NO HISTORY DEPENDENCE)
# ============================================================================
# This section demonstrates how to train a feedforward neural network (MLP)
# to approximate the stress–stretch relationship for a hyperelastic material.
# Since hyperelasticity is history-independent, a simple regression suffices.

# ----------------------------------------------------------------------------
# MODEL ARCHITECTURE
# ----------------------------------------------------------------------------
def build_mlp(hidden_layers=6, width=32, activation='tanh', l2_reg=None):
    '''
    Build a feedforward network (MLP) for regression.

    Key design notes:
    - hidden_layers: Controls model depth (number of hidden transformations)
    - width: Controls number of neurons per layer
    - activation: Controls nonlinearity (tanh gives smooth, symmetric response)
    - l2_reg: Optional regularization to penalize large weights (smoothens fit)

    For FAST mode, both depth and width are reduced for quick CPU testing.
    '''
    # Adjust model size based on FAST mode
    n_layers = max(2, hidden_layers // 2) if FAST else hidden_layers
    neurons = max(16, width // 2) if FAST else width

    model = keras.Sequential(name='hyperelastic_mlp')
    model.add(layers.Input(shape=(1,)))  # Input = stretch λ

    # Hidden layers (nonlinear transformations)
    for i in range(n_layers):
        model.add(layers.Dense(
            neurons,
            activation=activation,
            kernel_regularizer=regularizers.l2(l2_reg) if l2_reg else None,
            name=f'hidden_{i+1}'
        ))

    # Output: stress S (linear activation)
    model.add(layers.Dense(1, name='output'))
    return model


# ============================================================================
# MODEL INSTANTIATION AND DATA NORMALIZATION
# ============================================================================
# Normalization is crucial for stable training. Neural networks assume roughly
# standardized input/output scales (mean ~ 0, std ~ 1). Raw mechanical data
# may vary by orders of magnitude, so we normalize first.
# ----------------------------------------------------------------------------
model_clean = build_mlp()

mean_stretch = stretch_train.mean()
std_stretch = stretch_train.std()
mean_stress = stress_train.mean()
std_stress = stress_train.std()

X_norm = (stretch_train - mean_stretch) / std_stretch
y_norm = (stress_train - mean_stress) / std_stress


# ============================================================================
# MODEL COMPILATION
# ============================================================================
# Optimizer: Adam (adaptive learning rate, robust to noisy gradients)
# Loss: Mean Squared Error (MSE) — suitable for regression
# ----------------------------------------------------------------------------
model_clean.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='mse'
)


# ============================================================================
# TRAINING SETUP: EARLY STOPPING & CHECKPOINTING
# ============================================================================
# EarlyStopping prevents overfitting: stops training once validation loss stops
# improving. ModelCheckpoint saves the best-performing model to disk (.keras).
# ----------------------------------------------------------------------------
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath='model_best.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=0
    )
]


# ============================================================================
# TRAINING THE MODEL
# ============================================================================
# We train for up to 700 (FAST) or 1000 epochs. Validation split = 0.2 means
# 80% of data used for gradient updates, 20% for validation monitoring.
# ----------------------------------------------------------------------------
history = model_clean.fit(
    X_norm.reshape(-1, 1),
    y_norm.reshape(-1, 1),
    epochs=700 if FAST else 1000,
    verbose=0,
    validation_split=0.2
)


# ============================================================================
# MODEL EVALUATION AND VISUALIZATION
# ============================================================================
# We inspect:
# 1) Training curves (loss evolution)
# 2) Fit on training data
# 3) Extrapolation behavior beyond training range
# ----------------------------------------------------------------------------
y_pred = model_clean.predict(X_norm.reshape(-1, 1), verbose=0)
y_pred = y_pred.flatten() * std_stress + mean_stress  # Denormalize predictions

plt.figure(figsize=(12, 4))

# --- (1) Training evolution plot ---
plt.subplot(1, 3, 1)
plt.semilogy(history.history['loss'], label='Train')
plt.semilogy(history.history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Evolution')
plt.legend()
plt.grid(True, alpha=0.3)

# --- (2) Training data fit ---
plt.subplot(1, 3, 2)
plt.scatter(stretch_train, stress_train, s=100, alpha=0.7, label='True')
plt.scatter(stretch_train, y_pred, s=80, marker='x', color='red', label='Predicted')
plt.xlabel('Stretch λ [-]')
plt.ylabel('Stress S [kPa]')
plt.title('Training Fit')
plt.legend()
plt.grid(True, alpha=0.3)

# --- (3) Extrapolation test ---
stretch_test = np.linspace(0.7, 8.5, 200)
X_test_norm = (stretch_test - mean_stretch) / std_stretch
y_test_pred = model_clean.predict(X_test_norm.reshape(-1, 1), verbose=0)
y_test_pred = y_test_pred.flatten() * std_stress + mean_stress
y_test_true = compute_S_stress(stretch_test)

plt.subplot(1, 3, 3)
plt.plot(stretch_test, y_test_true, 'b-', label='True', linewidth=2)
plt.plot(stretch_test, y_test_pred, 'r--', label='NN', linewidth=2)
plt.axvspan(stretch_train.min(), stretch_train.max(), alpha=0.2, color='green')
plt.xlabel('Stretch λ [-]')
plt.ylabel('Stress S [kPa]')
plt.title('Extrapolation Test')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


# ============================================================================
# PERFORMANCE METRIC
# ============================================================================
# Compute mean absolute error (MAE) in original physical units (kPa)
# ----------------------------------------------------------------------------
mae_train = np.mean(np.abs(y_pred - stress_train))
print(f"\n✅ Training MAE: {mae_train:.3f} kPa")
print(f"⚠️  Extrapolation fails outside training domain!")


### Conceptual Checkpoint: Understanding vs Memorization

**Reflect on what you observed:**

1. **Loss behavior**: Did both losses decrease smoothly? When did validation loss stop improving?
2. **Fit quality**: How close are predictions to true values at training points?
3. **Interpolation**: Does the model produce smooth curves between training points?
4. **Extrapolation**: What happens outside the training domain?

**Key insights**:
- ✅ Excellent interpolation: NN creates smooth function through data points
- ✅ Efficient learning: Only needed 10-15 points to capture nonlinearity  
- ❌ Extrapolation failure: Predictions outside training domain are meaningless
- ❌ No physics constraints: Model doesn't know $S(1) = 0$ unless explicitly taught

**Engineering relevance**: This demonstrates both the power and danger of neural networks. They excel at interpolation but fail catastrophically at extrapolation. Always validate your training domain coverage!

---

## Level 2: Noisy Hyperelastic Data (Regularization and Overfitting)

### Why This Matters

**Real-world scenario**: Experimental measurements always contain noise from:
- Sensor limitations (± 0.1% full scale)
- Temperature fluctuations during testing
- Specimen imperfections and mounting errors
- Vibrations and electrical interference

**The challenge**: How do we learn the underlying physics without memorizing the noise?

### Learning Objectives

1. **Recognize overfitting**: Model fits training data perfectly but generalizes poorly
2. **Apply regularization**: L1, L2, dropout, and other techniques
3. **Use validation properly**: Monitor generalization during training
4. **Select hyperparameters**: Systematic approach to regularization strength

### Conceptual Foundation

**The bias-variance tradeoff**:
- **High bias** (underfitting): Model too simple, cannot capture true pattern
- **High variance** (overfitting): Model too complex, memorizes noise
- **Sweet spot**: Balance complexity with generalization

**Regularization techniques**:
- **L2 (weight decay)**: Penalizes large weights → smoother functions
- **L1 (sparsity)**: Pushes weights to zero → feature selection
- **Dropout**: Randomly disables neurons → ensemble effect
- **Early stopping**: Stop before overfitting → implicit regularization

### Prediction Before Running

What will happen when we train a large network on noisy data?
- Without regularization?
- With strong regularization?
- How can we tell if we have the right amount?

In [ ]:
# ============================================================================
# ADDING REALISTIC NOISE TO DATA
# ============================================================================
# Real experimental data is never perfect. We simulate realistic measurement noise.

# Noise level selection (15% is typical for material testing)
noise_level = 0.15  # Standard deviation as fraction of signal range

# Set seed for reproducible noise
np.random.seed(SEED)

# Add Gaussian noise to normalized stress values
# Why normalized? Keeps noise level consistent across all values
y_noisy_norm = y_norm + np.random.normal(0, noise_level, y_norm.shape)

# Convert back to physical units for visualization
stress_noisy = y_noisy_norm * std_stress + mean_stress

# ============================================================================
# VISUALIZE NOISY DATA VS TRUE FUNCTION
# ============================================================================
plt.figure(figsize=(10, 6))

# True function (what we want to learn)
x_true_curve = np.linspace(1, 6, 200)
y_true_curve = compute_S_stress(x_true_curve)
plt.plot(x_true_curve, y_true_curve, 'b-', linewidth=2.5,
        alpha=0.7, label='True function (unknown to model)')

# Noisy measurements (what the model actually sees)
plt.scatter(stretch_train, stress_noisy, s=100, color='red',
           edgecolors='black', linewidth=1.5,
           label=f'Noisy measurements (SNR = {1/noise_level:.1f})',
           zorder=5)

# Error bars to show noise level
plt.errorbar(stretch_train, stress_noisy,
            yerr=noise_level * std_stress,  # ±1 std of noise
            fmt='none', color='red', alpha=0.3, capsize=3)

plt.xlabel('Stretch λ [-]')
plt.ylabel('Stress S [kPa]')
plt.title('The Challenge: Learn the Signal, Ignore the Noise')
plt.legend()
plt.grid(True, alpha=0.3)

# Annotate signal-to-noise ratio
snr = std_stress / (noise_level * std_stress)
plt.text(0.02, 0.98, f'Signal-to-Noise Ratio: {snr:.1f}:1',
        transform=plt.gca().transAxes, fontsize=11,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"📊 Noise characteristics:")
print(f"   Noise level: ±{noise_level * std_stress:.1f} kPa (1σ)")
print(f"   Relative noise: {noise_level * 100:.0f}% of normalized range")
print(f"   SNR: {snr:.1f}:1")

## Demonstrating model capacity: Underfitting, good fit, and overfitting

Before applying neural networks to real material modeling problems, we must understand how model capacity affects prediction quality. This section demonstrates three archetypal scenarios using the hyperelastic stress-stretch relationship with noisy training data:

**Underfitting**: A model with insufficient capacity (too few parameters, excessive regularization) cannot capture the underlying function, even when provided with clean training data. The predictions remain biased and systematically deviate from the true relationship.

**Good fit**: A properly designed model with balanced capacity learns the underlying physical law while filtering out measurement noise. This represents the ideal scenario for material modeling applications.

**Overfitting**: An over-parameterized model with excessive capacity memorizes the training data, including noise and measurement artifacts. The resulting predictions exhibit spurious oscillations between data points and fail to generalize.

In [ ]:
# === COMPREHENSIVE DEMONSTRATION: Underfitting, Good Fit, and Overfitting ===

# ============================================================================
# PART 1: DATA NORMALIZATION (CRITICAL PREPROCESSING STEP)
# ============================================================================
# WHY NORMALIZE:
# Neural networks train best when inputs and outputs are on similar scales (typically -1 to 1 or 0 to 1).
# Our raw data: stretch λ ≈ 1-6, stress S ≈ 0-200 kPa (very different scales!)
# Without normalization: gradients become unstable, training fails or is very slow.
#
# HOW WE NORMALIZE (Z-score standardization):
# For any variable x: x_normalized = (x - mean(x)) / std(x)
# This transforms data to have mean ≈ 0 and standard deviation ≈ 1

# Calculate normalization parameters from TRAINING data only
# (Never use test data statistics, as that would be "cheating" - the model would peek at test data)
mean_stretch = stretch_train.mean()  # Average stretch value
std_stretch = stretch_train.std()    # Standard deviation of stretch
mean_stress = stress_noisy.mean()    # Average noisy stress value
std_stress = stress_noisy.std()      # Standard deviation of noisy stress

# Apply normalization: transform training data
X_norm = (stretch_train - mean_stretch) / std_stretch  # Input (stretch)
y_noisy_norm = (stress_noisy - mean_stress) / std_stress  # Output (noisy stress)

# Later, when making predictions, we'll denormalize: y_original = y_norm * std + mean

# ============================================================================
# PART 2: UNDERFITTING MODEL (TOO SIMPLE, TOO CONSTRAINED)
# ============================================================================
# GOAL: Demonstrate what happens when model capacity is insufficient

# ARCHITECTURE CHOICES:
# - Layers: 2 (very shallow - cannot learn complex patterns)
# - Neurons per layer: 8 (very narrow - limited representational power)
# - Total parameters: ~153 (10 data points could be fit perfectly, but model cannot)

# REGULARIZATION CHOICE:
# - L2 penalty = 0.1 (VERY STRONG)
#   L2 = "weight decay" = penalizes large weights
#   Loss = MSE + 0.1 * sum(weights²)
#   Effect: forces weights to stay small → smooth but biased predictions

# OPTIMIZER CHOICE:
# - SGD (Stochastic Gradient Descent) with learning_rate = 0.01
#   SGD = basic optimizer, updates weights in direction of negative gradient
#   Learning rate = step size (0.01 is slow, safe, but may underfit)
#   Alternative optimizers: Adam (adaptive, faster), RMSprop (adaptive)

print("Building underfitting model...")
model_under = build_mlp(hidden_layers=2, width=8, l2_reg=0.1)
model_under.compile(
    optimizer=keras.optimizers.SGD(0.01),  # Slow, basic optimizer
    loss="mse"  # Mean Squared Error = (prediction - truth)²
)
print(f"Underfitting model parameters: {model_under.count_params():,}")
print("Training underfitting model...")
hist_under = model_under.fit(
    X_norm[:,None],        # Input: normalized stretch ([:,None] reshapes to column vector)
    y_noisy_norm[:,None],  # Output: normalized noisy stress
    epochs=300,            # Number of times to see full dataset
    batch_size=len(X_norm),  # Full batch
    verbose=0,             # No output during training
    validation_split=0.2   # 20% validation for consistent comparison
)
print(f"  Completed: {len(hist_under.history['loss'])} epochs")

# ============================================================================
# PART 3: GOOD FIT MODEL (BALANCED CAPACITY, PROPER REGULARIZATION)
# ============================================================================
# GOAL: Learn true underlying function while ignoring noise

# ARCHITECTURE CHOICES:
# - Layers: 4 (enough depth to model nonlinear stress-strain relationship)
# - Neurons per layer: 64 (enough width for complex patterns, not excessive)
# - Activation: SWISH (smooth, avoids saturation, better than ReLU or tanh for this problem)
#
# ACTIVATION FUNCTIONS YOU CAN EXPERIMENT WITH:
# Replace "keras.activations.swish" with any of these:
#   - keras.activations.relu       # Fast, piecewise-linear (may create kinks)
#   - keras.activations.tanh       # Smooth, bounded [-1,1] (may saturate)
#   - keras.activations.sigmoid    # Smooth, bounded [0,1] (may saturate)
#   - keras.activations.elu        # Smooth negative values (good for deep nets)
#   - keras.activations.selu       # Self-normalizing (good for very deep nets)
#   - "linear"                     # No activation (not recommended for hidden layers)
#
# To experiment: Change line below and observe how middle panel curve changes!

def build_goodfit_mlp():
    """
    Build a feedforward neural network with balanced capacity.

    PYTHON PROGRAMMING EXPLANATION:
    - def build_goodfit_mlp(): defines a function (reusable block of code)
    - keras.Sequential: container for layers stacked one after another
    - layers.Input((1,)): defines input shape (1 number: stretch)
    - for _ in range(4): Python loop that repeats 4 times
    - layers.Dense(64, ...): fully-connected layer with 64 neurons
    - m.add(...): appends layer to the sequential model
    - return m: gives back the built model to whoever called this function
    """
    m = keras.Sequential([layers.Input((1,))])  # Create empty model, define input

    # Add 4 hidden layers (this loop executes 4 times)
    for layer_num in range(4):  # layer_num will be 0, 1, 2, 3
        m.add(layers.Dense(
            64,  # Number of neurons (nodes) in this layer
            activation=keras.activations.swish,  # ← EXPERIMENT HERE: try relu, tanh, elu
            kernel_regularizer=regularizers.l2(1e-3),  # L2 penalty (weight decay)
            name=f'hidden_{layer_num+1}'  # Name for debugging (hidden_1, hidden_2, ...)
        ))

    # Output layer: 1 neuron, linear activation (predicts stress value)
    m.add(layers.Dense(1, activation='linear', name='output'))

    # OPTIMIZER CHOICE:
    # Adam = Adaptive Moment Estimation
    #   - Adapts learning rate for each parameter individually
    #   - Combines momentum (smooths updates) and RMSprop (scales by gradient history)
    #   - learning_rate=1e-3 (0.001) is a good default
    #   - Generally faster and more robust than SGD
    m.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="mse"  # Mean Squared Error
    )
    return m

print("\nBuilding good fit model...")
model_good = build_goodfit_mlp()
print(f"Good fit model parameters: {model_good.count_params():,}")

# TRAINING WITH EARLY STOPPING:
# Early stopping = stop training when validation loss stops improving
# This prevents overfitting: we stop before the model starts memorizing noise
print("Training good fit model with early stopping...")
hist_good = model_good.fit(
    X_norm[:,None], y_noisy_norm[:,None],
    epochs=2000,  # Maximum epochs (early stopping may stop sooner)
    batch_size=len(X_norm),  # Full batch: use all data per update (stable gradients)
    verbose=0,  # No output (clean console)
    validation_split=0.2,  # Use 20% of data for validation (2 points), 80% for training (8 points)
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss",  # Watch validation loss (not training loss)
            patience=120,  # Wait 120 epochs after best val_loss before stopping
            restore_best_weights=True,  # Revert to epoch with best val_loss (not last)
            verbose=0  # No output
        )
    ]
)
print(f"  Completed: {len(hist_good.history['loss'])} epochs (early stopping)")

# ============================================================================
# PART 4: OVERFITTING MODEL (EXTREME CAPACITY, NO REGULARIZATION)
# ============================================================================
# GOAL: Memorize every training point, including noise

# ARCHITECTURE CHOICES:
# - Layers: 8 (very deep - can create complex interpolations)
# - Neurons per layer: 256 (very wide - massive capacity)
# - Activation: ReLU (creates piecewise-linear functions, enables sharp kinks)
# - Total parameters: ~526,000 (for 10 data points!)

# TRAINING STRATEGY:
# - batch_size = 1 (SGD: update after EACH point, not after all points)
# - shuffle = True (randomize order every epoch - helps memorization)
# - epochs = 3000-6000 (excessive training time - learns noise as signal)
# - NO regularization (L2 = 0) - nothing prevents memorization
# - validation_split = 0.2 to show gap between train and val loss

# Train on PHYSICAL units (not normalized) to exaggerate wiggles
y_noisy_phys = stress_noisy  # Use original noisy stress values

def build_overfit_mlp():
    """
    Build an intentionally over-parameterized network to demonstrate overfitting.

    PYTHON PROGRAMMING EXPLANATION:
    - This function uses the same structure as build_goodfit_mlp()
    - Key difference: MORE layers (8 vs 4), MORE neurons (256 vs 64)
    - No regularization: kernel_regularizer is omitted (defaults to None)
    """
    m = keras.Sequential([layers.Input((1,))])

    # Add 8 hidden layers with 256 neurons each (massive capacity!)
    for layer_num in range(8):
        m.add(layers.Dense(
            256,  # 256 neurons per layer
            activation="relu",  # ReLU = max(0, x), creates piecewise-linear kinks
            name=f'overfit_hidden_{layer_num+1}'
        ))
        # Note: NO kernel_regularizer - nothing penalizes large weights!

    m.add(layers.Dense(1, activation='linear', name='output'))
    m.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")
    return m

print("\nBuilding overfitting model...")
model_over = build_overfit_mlp()
print(f"Overfitting model parameters: {model_over.count_params():,} (vs {len(X_norm)} data points!)")

print("Training overfitting model (memorizing noise)...")
hist_over = model_over.fit(
    X_norm[:, None], y_noisy_phys[:, None],
    epochs=6000 if not FAST else 3000,
    batch_size=1,  # SGD on individual points (not batches)
    verbose=0,  # No output (clean console)
    shuffle=True,  # Randomize point order each epoch
    validation_split=0.2  # Track validation loss to show train/val gap
)
print(f"  Completed: {len(hist_over.history['loss'])} epochs")

# ============================================================================
# PART 5: EVALUATION AND VISUALIZATION
# ============================================================================

# Create dense grid for smooth prediction curves
stretch_dense = np.linspace(stretch_train.min(), stretch_train.max(), 500)
X_dense_norm = (stretch_dense - mean_stretch) / std_stretch  # Normalize using training statistics

# Ground truth (analytical solution)
stress_true_dense = compute_S_stress(stretch_dense)

# Model predictions (denormalize outputs back to physical units)
pred_under = model_under.predict(X_dense_norm[:,None], verbose=0).ravel() * std_stress + mean_stress
pred_good  = model_good.predict(X_dense_norm[:,None], verbose=0).ravel() * std_stress + mean_stress
pred_over  = model_over.predict(X_dense_norm[:,None], verbose=0).ravel()  # Already in physical units

stress_noisy = y_noisy_norm * std_stress + mean_stress  # Denormalize noisy training data

# Plot three-panel comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
titles = ["Underfitting\n(Too simple / heavy regularization)",
          "Good fit\n(Balanced capacity + L2)",
          "Overfitting\n(Memorizes noise)"]
curves  = [pred_under, pred_good, pred_over]
colors  = ["#ff7f0e", "#2ca02c", "#9467bd"]  # Orange, green, purple

# Set common y-limits for fair comparison
y_min = min(stress_true_dense.min(), stress_noisy.min()) - 5
y_max = max(stress_true_dense.max(), stress_noisy.max()) + 5

true_color = "#1f77b4"  # Force true curve to blue in all panels

for ax, pred, title, col in zip(axes, curves, titles, colors):
    # True function (always blue for consistency)
    ax.plot(stretch_dense, stress_true_dense, lw=2, alpha=0.8,
            color=true_color, label="True function")
    # Model prediction (panel-specific color)
    ax.plot(stretch_dense, pred, lw=2.5, color=col, label="Model prediction")
    # Noisy training points (drawn on top, visible)
    ax.scatter(
        stretch_train, stress_noisy, s=50, color="crimson",
        edgecolors="black", linewidth=0.5, alpha=0.9, zorder=5,
        label="Noisy data"
    )
    ax.set_xlabel(r"Stretch $\lambda$ [–]")
    ax.set_ylabel(r"Stress $S$ [kPa]")
    ax.set_title(title)
    ax.set_ylim(y_min, y_max)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="lower right", fontsize=9)

plt.suptitle("Underfitting vs. Good Fit vs. Overfitting", fontsize=15, fontweight="bold")
plt.tight_layout()
plt.show()

# Print capacity summary
print(f"\nModel capacity comparison:")
print(f"  Underfitting:  {model_under.count_params():>7,} parameters")
print(f"  Good fit:      {model_good.count_params():>7,} parameters")
print(f"  Overfitting:   {model_over.count_params():>7,} parameters")
print(f"  Training data: {len(stretch_train):>7} points")
print(f"  Parameter/data ratio (overfit): {model_over.count_params()/len(stretch_train):,.0f}:1")

## Level 3: Viscoelastic material (history-dependent)

### Why this matters

**Engineering challenge**: Many materials exhibit memory effects that cannot be captured by instantaneous stress-stretch relationships. Viscoelastic polymers under cyclic loading demonstrate three critical phenomena:

- **Rate dependence**: Faster loading rates produce stiffer mechanical responses and higher peak stresses
- **Hysteresis**: The loading and unloading paths differ, creating closed loops in stress-stretch space
- **Energy dissipation**: The area enclosed by the hysteresis loop represents mechanical energy converted to heat

Traditional modeling approaches require solving coupled differential equations at each material point. The question arises: can neural networks learn this complex history-dependent behavior directly from experimental data?

### The fundamental problem

**Feedforward networks are fundamentally inadequate** because they learn mappings of the form:

$$S = f(\lambda)$$

This assumes stress depends only on the current stretch state. However, viscoelastic behavior constitutes a **functional** relationship:

$$S(t) = \mathcal{F}\left[\{\lambda(\tau), \dot{\lambda}(\tau)\}_{\tau \leq t}\right]$$

The current stress state depends on the entire loading history up to the present time, not merely the instantaneous stretch. This history dependence manifests in two critical ways:

1. **Path dependence**: Two specimens stretched to the same $\lambda$ will exhibit different stresses if they followed different loading histories
2. **Rate sensitivity**: The same stretch amplitude applied at different rates produces different stress responses

### The Maxwell power-law constitutive model

The code implements a nonlinear Maxwell model combining elastic and viscous elements in series. The governing differential equation is:

$$\frac{d\sigma}{dt} = E \frac{d\varepsilon}{dt} - \frac{\sigma}{\eta_0} \left|\frac{\sigma}{\sigma_0}\right|^{m-1}$$

where the material parameters are:

- $E = 3.0$ kPa: elastic modulus (instantaneous stiffness)
- $\eta_0 = 0.2$ kPa·s: viscosity coefficient (resistance to flow)
- $\sigma_0 = 1.0$ kPa: reference stress (normalization constant)
- $m = 3$: power-law exponent (degree of nonlinearity)

**Physical interpretation of the terms**:

1. **Elastic term** $E \frac{d\varepsilon}{dt}$: Represents instantaneous stress buildup proportional to the strain rate. This captures the elastic spring behavior of the material.

2. **Viscous term** $\frac{\sigma}{\eta_0} \left|\frac{\sigma}{\sigma_0}\right|^{m-1}$: Describes time-dependent stress relaxation through viscous flow. The power-law form captures nonlinear viscosity, allowing the model to represent shear thinning ($m < 1$) or shear thickening ($m > 1$) behavior.

### Observable phenomena in the simulation results

The generated hysteresis loops reveal several key characteristics:

1. **Rate-dependent stiffening**: The fast loading curve ($\dot{\lambda} = 3.0$ s$^{-1}$) reaches significantly higher peak stresses than the slow curve, demonstrating viscous resistance to rapid deformation.

2. **Hysteresis loop area**: The enclosed area represents energy dissipated per loading cycle. Faster rates typically produce larger loops, indicating greater energy dissipation.

3. **Stress relaxation during unloading**: The unloading path lies below the loading path because stress has time to relax through viscous flow.

4. **Non-equilibrium behavior**: At any given stretch $\lambda$, the stress state depends on whether the material is currently loading or unloading and at what rate.

### Learning objectives

Working with this viscoelastic model teaches several critical concepts for neural network applications:

1. **Recognize when recurrent architectures are necessary**: Any phenomenon exhibiting path dependence, rate sensitivity, or memory effects requires sequential models rather than feedforward networks.

2. **Format sequential data properly**: Time series must be structured with appropriate features (stretch, stretch rate, time) and target variables (stress). The temporal ordering must be preserved.

3. **Understand hidden states in RNNs**: The hidden state vector serves as a learned representation of the loading history, effectively replacing the need to explicitly track $\{\lambda(\tau), \dot{\lambda}(\tau)\}_{\tau \leq t}$.

4. **Validate generalization to new loading paths**: The trained RNN must be tested on loading rates not seen during training to assess whether it has learned the underlying physics rather than memorizing specific trajectories.

In [ ]:
# ============================================================================
# VISCOELASTIC MATERIAL MODEL: MAXWELL POWER-LAW
# ============================================================================
# This model captures both elastic (instant) and viscous (time-dependent) response

# Material parameters (typical for viscoelastic polymer)
E_visco = 3.0      # Elastic modulus [kPa] - instant stiffness
nu_0 = 2e-1        # Viscosity coefficient [kPa·s] - flow resistance
sigma_0 = 1.0      # Reference stress [kPa] - normalizes nonlinearity
m = 3              # Power-law exponent [-] - degree of nonlinearity

def sigma_dot(sigma, strain_rate):
    """
    Compute stress rate from Maxwell power-law constitutive model.

    The differential equation is:
    dσ/dt = E·dε/dt - (σ/η)·|σ/σ₀|^(m-1)

    Physical interpretation:
    - First term: Elastic response (instant, proportional to strain rate)
    - Second term: Viscous relaxation (time-dependent stress decay)
    - Power law: Captures nonlinear viscosity (shear thinning/thickening)

    Parameters:
        sigma: Current Cauchy stress [kPa]
        strain_rate: Current strain rate dε/dt [1/s]

    Returns:
        dsigma_dt: Rate of stress change [kPa/s]
    """
    # Elastic contribution: stress builds proportionally to strain rate
    elastic_term = E_visco * strain_rate

    # Viscous contribution: stress relaxes over time
    if abs(sigma) < 1e-10:  # Avoid division by zero at zero stress
        viscous_term = 0.0
    else:
        # Power-law viscosity: resistance depends on stress level
        viscous_term = (sigma / nu_0) * abs(sigma / sigma_0)**(m - 1)

    # Total rate: build-up minus relaxation
    return elastic_term - viscous_term

# ============================================================================
# GENERATE REFERENCE VISCOELASTIC CURVES
# ============================================================================

def generate_visco_path(rate, dt_factor=1.0):
    """
    Generate viscoelastic loading/unloading cycle.

    Simulates a complete cycle:
    1. Loading: Stretch from λ=1.0 to λ=2.5 at constant rate
    2. Unloading: Return to λ=1.0 at same rate

    This creates a hysteresis loop showing energy dissipation.

    Parameters:
        rate: Strain rate [1/s] - how fast we stretch
        dt_factor: Time step multiplier for numerical stability

    Returns:
        stretches, stresses, times, rates: Arrays of the loading path
    """
    # Time step: smaller for higher rates (numerical stability)
    dt = 1e-3 / rate * dt_factor

    # Initialize state
    stretch = 1.0  # Start undeformed
    stress = 0.0   # Start stress-free

    # Storage arrays
    stretches = []
    stresses = []
    times = []
    rates = []

    t = 0.0

    # PHASE 1: Loading (stretch increases)
    while stretch < 2.5:
        # Record current state
        stretches.append(stretch)
        stresses.append(stress / stretch**2)  # Convert Cauchy to S
        times.append(t)
        rates.append(rate)

        # Integrate one time step (Forward Euler)
        dsigma_dt = sigma_dot(stress, rate)
        stretch += dt * rate      # Update stretch
        stress += dt * dsigma_dt  # Update stress
        t += dt

    # PHASE 2: Unloading (stretch decreases)
    while stretch > 1.01:  # Stop near λ=1
        stretches.append(stretch)
        stresses.append(stress / stretch**2)
        times.append(t)
        rates.append(-rate)  # Negative rate for unloading

        dsigma_dt = sigma_dot(stress, -rate)
        stretch -= dt * rate
        stress += dt * dsigma_dt
        t += dt

    return np.array(stretches), np.array(stresses), np.array(times), np.array(rates)

# ============================================================================
# GENERATE TWO REFERENCE CURVES AT DIFFERENT RATES
# ============================================================================
print("🔄 Generating viscoelastic reference curves...")

# Two different loading rates to show rate dependence
rate1 = 1.0  # Slow loading [1/s]
rate2 = 3.0  # Fast loading [1/s]

# Generate complete loading cycles
stretch1, stress1, time1, rates1 = generate_visco_path(rate1)
stretch2, stress2, time2, rates2 = generate_visco_path(rate2)

print(f"✅ Generated reference curves:")
print(f"   Curve 1: rate = {rate1}/s, {len(stretch1)} points, duration = {time1[-1]:.2f}s")
print(f"   Curve 2: rate = {rate2}/s, {len(stretch2)} points, duration = {time2[-1]:.2f}s")

# ============================================================================
# VISUALIZE VISCOELASTIC BEHAVIOR
# ============================================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# LEFT PANEL: Stress-stretch curves (hysteresis loops)
ax1.plot(stretch1, stress1, 'b-', linewidth=2.5,
        label=f'Slow (rate = {rate1}/s)', alpha=0.8)
ax1.plot(stretch2, stress2, 'r-', linewidth=2.5,
        label=f'Fast (rate = {rate2}/s)', alpha=0.8)

# Mark loading direction with arrows
mid_idx1 = len(stretch1) // 4
ax1.annotate('', xy=(stretch1[mid_idx1], stress1[mid_idx1]),
            xytext=(stretch1[mid_idx1-5], stress1[mid_idx1-5]),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))

mid_idx2 = len(stretch2) // 4
ax1.annotate('', xy=(stretch2[mid_idx2], stress2[mid_idx2]),
            xytext=(stretch2[mid_idx2-5], stress2[mid_idx2-5]),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))

ax1.set_xlabel('Stretch λ [-]')
ax1.set_ylabel('Second Piola-Kirchhoff Stress S [kPa]')
ax1.set_title('Viscoelastic Hysteresis Loops')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add annotation about energy dissipation
ax1.fill(stretch1, stress1, alpha=0.1, color='blue')
ax1.text(1.75, 10, 'Area = Energy\ndissipated',
        ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# RIGHT PANEL: Time evolution
ax2.plot(time1, stretch1, 'b-', linewidth=1.5,
        label='Stretch (slow)', alpha=0.7)
ax2.plot(time1, stress1, 'b--', linewidth=1.5,
        label='Stress (slow)', alpha=0.7)
ax2.plot(time2, stretch2, 'r-', linewidth=1.5,
        label='Stretch (fast)', alpha=0.7)
ax2.plot(time2, stress2, 'r--', linewidth=1.5,
        label='Stress (fast)', alpha=0.7)

ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Value')
ax2.set_title('Time Series: Loading and Unloading')
ax2.legend(fontsize=9, loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# KEY OBSERVATIONS TO UNDERSTAND
# ============================================================================
print(f"\n🔑 Critical observations:")
print(f"   1. Rate dependence: Faster loading → Higher peak stress")
print(f"   2. Hysteresis: Different paths for loading vs unloading")
print(f"   3. Energy dissipation: Area inside loop = energy lost")
print(f"   4. History matters: Same stretch, different stress depending on path")
print(f"\n⚠️  Feedforward networks CANNOT capture this behavior!")

### Recurrent neural networks

**Recurrent neural networks (RNNs)** resolve the history-dependence problem through an internal memory mechanism. Unlike feedforward networks, RNNs maintain a **hidden state vector** $\mathbf{h}_t$ that evolves over time:

$$\mathbf{h}_t = \phi(\mathbf{W}_h \mathbf{h}_{t-1} + \mathbf{W}_x \mathbf{x}_t + \mathbf{b}_h)$$

$$\mathbf{y}_t = \mathbf{W}_y \mathbf{h}_t + \mathbf{b}_y$$

where:
- $\mathbf{x}_t = [\lambda_t, \dot{\lambda}_t, \Delta t]^T$ is the input feature vector at time $t$
- $\mathbf{h}_t$ encodes the complete loading history up to time $t$
- $\mathbf{y}_t = S_t$ is the predicted stress output
- $\mathbf{W}_h, \mathbf{W}_x, \mathbf{W}_y$ are learnable weight matrices
- $\phi(\cdot)$ is a nonlinear activation function (typically tanh or ReLU)

**How RNNs capture material memory**: The hidden state $\mathbf{h}_t$ serves as a compressed representation of all previous loading history $\{\lambda(\tau), \dot{\lambda}(\tau)\}_{\tau < t}$. At each time step, the network:

1. Receives current kinematic input $[\lambda_t, \dot{\lambda}_t, \Delta t]$
2. Updates the hidden state by combining new information with previous history
3. Predicts stress $S_t$ based on the updated hidden state

This architecture naturally respects causality because $\mathbf{h}_t$ only depends on inputs from $\tau \leq t$, never future values.

**Key advantages for viscoelastic modeling**:
- **No explicit history storage**: The network learns optimal compression of past information
- **Automatic feature extraction**: No need to manually engineer history-dependent features
- **Generalizable**: Can predict stress for arbitrary loading paths not seen during training


In [ ]:
# ============================================================================
# RNN ARCHITECTURE FOR VISCOELASTIC MODELING
# ============================================================================

def build_simple_rnn(hidden_size=8, num_layers=2):
    """
    Build a Recurrent Neural Network for history-dependent material modeling.

    KEY CONCEPT: RNNs maintain a hidden state that summarizes history:
    h_t = f(x_t, h_{t-1})
    y_t = g(h_t)

    This allows the network to "remember" past loading and make predictions
    based on the entire history, not just current input.

    ARCHITECTURE DECISIONS:
    1. Hidden size: Dimensionality of memory vector
       - Small (4-8): Forces compression, may lose details
       - Large (32-64): Rich memory but may overfit

    2. Number of layers: Depth of temporal processing
       - 1 layer: Simple memory, fast
       - 2-3 layers: Hierarchical temporal features
       - >3 layers: Risk of gradient vanishing

    3. Input features: [stretch, strain_rate, dt]
       - stretch: Current deformation state
       - strain_rate: How fast we're loading
       - dt: Time step (for numerical integration)

    Parameters:
        hidden_size: Size of RNN hidden state
        num_layers: Number of stacked RNN layers

    Returns:
        Keras model with RNN architecture
    """

    # Adjust for FAST mode (CPU-friendly)
    h_size = hidden_size if not FAST else max(4, hidden_size // 2)
    n_layers = num_layers if not FAST else max(1, num_layers // 2)

    # Define input: sequences of variable length
    # Shape: (batch_size, time_steps, 3)
    # The 3 features are: [stretch, strain_rate, dt]
    inp = layers.Input(shape=(None, 3), name='sequence_input')

    x = inp

    # Stack RNN layers
    for i in range(n_layers):
        # All layers return sequences (we need per-timestep predictions)
        x = layers.SimpleRNN(
            h_size,
            activation='tanh',      # Smooth, bounded activation
            return_sequences=True,  # Output at every time step
            name=f'rnn_layer_{i+1}'
        )(x)

        # How it works internally:
        # For each time step t:
        #   h[t] = tanh(W_h @ h[t-1] + W_x @ x[t] + b)
        #   output[t] = h[t]

    # Output layer: predict stress at each time step
    out = layers.Dense(1, activation='linear', name='stress_output')(x)

    # Build model
    model = keras.Model(inp, out, name='viscoelastic_rnn')

    print(f"\n📊 RNN Architecture (FAST={'ON' if FAST else 'OFF'}):")
    print(f"   Hidden state size: {h_size}")
    print(f"   Number of RNN layers: {n_layers}")
    print(f"   Total parameters: {model.count_params():,}")
    print(f"   Input: (batch, timesteps, 3) → [stretch, rate, dt]")
    print(f"   Output: (batch, timesteps, 1) → [stress at each time]")

    return model

# ============================================================================
# BUILD AND VISUALIZE RNN
# ============================================================================
print("\n🏗️ Building RNN for viscoelastic modeling...")

hidden_state_size = 8 if not FAST else 4
rnn_layers = 3 if not FAST else 2

model_rnn_visco = build_simple_rnn(
    hidden_size=hidden_state_size,
    num_layers=rnn_layers
)

# Show detailed architecture
model_rnn_visco.summary()


The critical challenge becomes preparing sequential training data that captures the essential physics of rate-dependent, hysteretic behavior.

### Generating variable-rate training sequences

The training data generation strategy differs fundamentally from the hyperelastic and elastoplastic cases. Rather than sampling random points in stretch space, we must generate **temporal sequences** that capture the history-dependent nature of viscoelastic response.

### Feature engineering for temporal learning

**Input feature vector**: At each time step $t$, the RNN receives three features:

$$\mathbf{x}_t = \begin{bmatrix} \lambda_t \\ \dot{\lambda}_t \\ \Delta t \end{bmatrix}$$

**Physical justification for each feature**:

1. **Stretch $\lambda_t$**: Current deformation state (kinematic variable)
2. **Stretch rate $\dot{\lambda}_t$**: Rate of deformation (captures viscous effects)
3. **Time increment $\Delta t$**: Duration of the current step (necessary for numerical integration of rate-dependent processes)

The inclusion of $\Delta t$ is crucial because the stress evolution depends not only on the rate magnitude but also on the duration over which that rate is applied. Two scenarios with identical $\lambda_t$ and $\dot{\lambda}_t$ but different $\Delta t$ values will produce different stress increments due to viscous relaxation effects.

**Output target**: The second Piola-Kirchhoff stress $S_t = \sigma_t / \lambda_t^2$ at each time step.

**Tensor shape requirements**: RNN layers expect input tensors with shape:

$$(B, T, F)$$

where:
- $B$ = batch size (number of independent sequences)
- $T$ = sequence length (number of time steps)
- $F$ = feature dimension (number of input variables)

For this viscoelastic training:
- **Input**: `X_rnn.shape = (1, T, 3)` where $T \approx 500$ time steps, 3 features
- **Output**: `y_rnn.shape = (1, T, 1)` where we predict 1 stress value per time step

**Single sequence training**: The batch size $B=1$ indicates we train on a single long temporal sequence. This approach is appropriate for the current problem because:

1. The sequence is sufficiently long ($T \sim 500$) to capture multiple loading-unloading transitions
2. Variable rates within the sequence provide diverse training examples
3. Temporal dependencies within one sequence are more important than batch diversity

In production applications with multiple experimental specimens, one would increase batch size by stacking multiple independent loading histories.

In [ ]:
# ============================================================================
# PREPARE TRAINING DATA FOR RNN
# ============================================================================
print("\n📦 Preparing sequential data for RNN training...")

# Generate training sequence with variable loading rates
# This teaches the network to handle different loading conditions
np.random.seed(SEED)

# Initialize
stretch_seq = []
stress_seq = []
rate_seq = []
time_seq = []

stretch = 1.0
stress = 0.0
t = 0.0

# Parameters for variable-rate loading
rate_mean = 1.2  # Average strain rate [1/s]
dt_mean = 1e-1 / rate_mean

print("   Generating variable-rate loading history...")

# LOADING PHASE: Stretch from 1.0 to 2.5 with varying rates
while stretch < 2.5:
    stretch_seq.append(stretch)
    stress_seq.append(stress / stretch**2)  # Convert to S

    # Randomly vary the time step and rate (±20% variation)
    dt = dt_mean * np.random.uniform(0.8, 1.2)
    rate = rate_mean * np.random.uniform(0.8, 1.2)

    rate_seq.append(rate)
    time_seq.append(t)

    # Integrate constitutive equation
    dsigma_dt = sigma_dot(stress, rate)
    stretch += dt * rate
    stress += dt * dsigma_dt
    t += dt

# UNLOADING PHASE: Return to λ=1 with varying rates
while stretch > 1.01:
    stretch_seq.append(stretch)
    stress_seq.append(stress / stretch**2)

    dt = dt_mean * np.random.uniform(0.8, 1.2)
    rate = -rate_mean * np.random.uniform(0.8, 1.2)

    rate_seq.append(rate)
    time_seq.append(t)

    dsigma_dt = sigma_dot(stress, rate)
    stretch += dt * rate
    stress += dt * dsigma_dt
    t += dt

# Convert to arrays
stretch_seq = np.array(stretch_seq)
stress_seq = np.array(stress_seq)
rate_seq = np.array(rate_seq)
time_seq = np.array(time_seq)

print(f"   Generated sequence: {len(stretch_seq)} time steps")

# ============================================================================
# FORMAT DATA FOR RNN
# ============================================================================
print("\n🔧 Formatting data for RNN (normalization + feature stacking)...")

# Compute time steps
dt_seq = np.diff(time_seq, prepend=time_seq[0])
if len(time_seq) > 1 and dt_seq[0] == 0:
    dt_seq[0] = float(np.median(np.diff(time_seq)))

# Safe normalization (avoid division by zero)
def safe_mean_std(x, eps=1e-12):
    m = float(np.mean(x))
    s = float(np.std(x))
    if s < eps:
        s = 1.0  # No scaling if variance is ~0
    return m, s

# Calculate normalization statistics
mean_rate, std_rate = safe_mean_std(rate_seq)
mean_dt, std_dt = safe_mean_std(dt_seq)
mean_str, std_str = safe_mean_std(stretch_seq)
mean_st, std_st = safe_mean_std(stress_seq)

print(f"   Normalization statistics:")
print(f"     Stretch: μ={mean_str:.2f}, σ={std_str:.2f}")
print(f"     Stress:  μ={mean_st:.1f}, σ={std_st:.1f}")
print(f"     Rate:    μ={mean_rate:.2f}, σ={std_rate:.2f}")

# Normalize all features
stretch_norm = (stretch_seq - mean_str) / std_str
rate_norm = (rate_seq - mean_rate) / std_rate
dt_norm = (dt_seq - mean_dt) / std_dt
stress_norm = (stress_seq - mean_st) / std_st

# Stack features: each time step has [stretch, rate, dt]
X_rnn = np.stack([stretch_norm, rate_norm, dt_norm], axis=1)  # Shape: (T, 3)
y_rnn = stress_norm.reshape(-1, 1)                            # Shape: (T, 1)

# Add batch dimension (RNNs expect batched data)
X_rnn = X_rnn[np.newaxis, :, :]  # Shape: (1, T, 3)
y_rnn = y_rnn[np.newaxis, :, :]  # Shape: (1, T, 1)

print(f"   RNN input shape: {X_rnn.shape} → (batch=1, time={X_rnn.shape[1]}, features=3)")
print(f"   RNN output shape: {y_rnn.shape} → (batch=1, time={y_rnn.shape[1]}, outputs=1)")

# ============================================================================
# VISUALIZE TRAINING SEQUENCE
# ============================================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# LEFT: Stress-stretch with color-coded time
scatter = ax1.scatter(stretch_seq, stress_seq,
                     s=20, c=time_seq, cmap='viridis',
                     alpha=0.6, edgecolors='none')
ax1.set_xlabel('Stretch λ [-]')
ax1.set_ylabel('Second Piola-Kirchhoff Stress S [kPa]')
ax1.set_title('Training Sequence (color = time progression)')
ax1.grid(True, alpha=0.3)
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Time [s]')

# RIGHT: Time series
ax2.plot(time_seq, stretch_seq, 'b-', linewidth=1, label='Stretch', alpha=0.7)
ax2.plot(time_seq, stress_seq, 'r-', linewidth=1, label='Stress', alpha=0.7)
ax2.plot(time_seq, rate_seq, 'g--', linewidth=0.8, label='Rate', alpha=0.5)
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Value')
ax2.set_title('Training Time Series')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Architecture summary**:
```
Input: (batch, time, features=3) → RNN(32) → Dense(1) → Output: (batch, time, outputs=1)
```

The 32-unit RNN provides sufficient capacity to learn the viscoelastic dynamics without overfitting to the training sequence. Smaller networks (e.g., 16 units) may underfit, while larger networks (e.g., 64+ units) risk memorizing the specific training trajectory rather than learning the underlying constitutive law.

In [ ]:
# ============================================================================
# TRAIN RNN WITH PROGRESS MONITORING
# ============================================================================
print("\n🏋️ Training RNN on viscoelastic sequence...")

# Configure optimizer
opt = keras.optimizers.Adam(learning_rate=1e-2 if not FAST else 5e-3)
model_rnn_visco.compile(optimizer=opt, loss="mse", metrics=["mae"])

# Callbacks for training
es_rnn = callbacks.EarlyStopping(
    patience=200 if not FAST else 50,
    restore_best_weights=True,
    monitor="loss",
    verbose=0
)

# Checkpoint callback (Keras 3 compatible)
ckpt_rnn = callbacks.ModelCheckpoint(
    filepath="model_best_rnn_visco.weights.h5",
    monitor="loss",
    mode="min",
    save_best_only=True,
    save_weights_only=True,  # Keras 3 requirement
    verbose=0,
)

# ============================================================================
# CUSTOM PROGRESS BAR CALLBACK
# ============================================================================
from tqdm.keras import TqdmCallback

# Create progress bar callback
tqdm_callback = TqdmCallback(verbose=0)

# Combine all callbacks
all_callbacks = [es_rnn, ckpt_rnn, tqdm_callback]

# Train the model with progress bar
print("   Training with early stopping and progress monitoring...")
history_rnn_visco = model_rnn_visco.fit(
    X_rnn, y_rnn,
    epochs=(2000 if not FAST else 500),
    batch_size=1,
    verbose=0,  # Suppress default output (tqdm handles it)
    callbacks=all_callbacks,
    shuffle=False  # Keep sequence order!
)

# Save final model
model_rnn_visco.save("model_rnn_visco_final.keras")

print(f"\n✅ Training complete!")
print(f"   Stopped at epoch: {len(history_rnn_visco.history['loss'])}")
print(f"   Final loss: {history_rnn_visco.history['loss'][-1]:.2e}")
print(f"   Final MAE: {history_rnn_visco.history['mae'][-1]:.3f}")
print(f"   Models saved: 'model_best_rnn_visco.weights.h5' and 'model_rnn_visco_final.keras'")

# ============================================================================
# PLOT TRAINING HISTORY
# ============================================================================
fig, ax = plt.subplots(figsize=(8, 5))
ax.semilogy(history_rnn_visco.history['loss'], 'b-', linewidth=2,
           label='Training loss', alpha=0.8)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss (log scale)')
ax.set_title('RNN Training on Viscoelastic Data')
ax.legend()
ax.grid(True, alpha=0.3)

# Mark early stopping point
min_loss_epoch = np.argmin(history_rnn_visco.history['loss'])
ax.axvline(min_loss_epoch, color='red', linestyle='--', alpha=0.5,
          label=f'Best model (epoch {min_loss_epoch})')

plt.tight_layout()
plt.show()

# ============================================================================
# EVALUATE RNN ON TRAINING SEQUENCE
# ============================================================================
print("\n📈 Evaluating RNN performance on training data...")

# Make predictions
y_pred_rnn = model_rnn_visco.predict(X_rnn, verbose=0)

# Denormalize to physical units
y_pred_rnn_physical = y_pred_rnn[0, :, 0] * std_st + mean_st

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Panel 1: Hysteresis loop reconstruction
ax = axes[0]
ax.scatter(stretch_seq, stress_seq, s=40, alpha=0.6, c='blue',
          label='True', zorder=3)
ax.scatter(stretch_seq, y_pred_rnn_physical, s=30, alpha=0.7, marker='x',
          c='red', label='RNN prediction', zorder=4)
ax.set_xlabel('Stretch λ [-]')
ax.set_ylabel('Second Piola-Kirchhoff Stress S [kPa]')
ax.set_title('RNN Captures Hysteresis Loop')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Time series comparison
ax = axes[1]
ax.plot(time_seq, stress_seq, 'b-', linewidth=2,
       label='True', alpha=0.7)
ax.plot(time_seq, y_pred_rnn_physical, 'r--', linewidth=2,
       label='RNN prediction', alpha=0.8)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Stress S [kPa]')
ax.set_title('Time Series Prediction')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: Residuals colored by rate
ax = axes[2]
residuals_rnn = y_pred_rnn_physical - stress_seq
scatter = ax.scatter(time_seq, residuals_rnn, s=30, alpha=0.6,
                    c=rate_seq, cmap='coolwarm')
ax.axhline(0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Residual [kPa]')
ax.set_title('Prediction Error (color = strain rate)')
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Rate [1/s]')

# Set symmetric y-limits for residuals
max_res = max(abs(residuals_rnn.min()), abs(residuals_rnn.max()))
ax.set_ylim(-max_res*1.1, max_res*1.1)

plt.tight_layout()
plt.show()

# Quantitative metrics
mae_rnn_train = np.mean(np.abs(residuals_rnn))
rmse_rnn_train = np.sqrt(np.mean(residuals_rnn**2))

print(f"\n📊 RNN performance on training sequence:")
print(f"   MAE: {mae_rnn_train:.3f} kPa")
print(f"   RMSE: {rmse_rnn_train:.3f} kPa")
print(f"   ✅ Successfully captures hysteresis and rate dependence!")
print(f"   ✅ The hidden state maintains memory of loading history")

---

## Level 4: Sparse Identification of Nonlinear Dynamics (SINDy)

### The Interpretability Challenge

We have seen that RNNs can learn history-dependent behavior, but they are **black boxes**. The hidden state contains information about history, but what exactly does it encode? What are the actual governing equations?

**SINDy offers a different approach**: Instead of learning a black-box function approximator, discover the actual differential equations that govern the system.

### Why SINDy Matters for Engineering

**Traditional approach**: Derive equations from first principles (conservation laws, constitutive assumptions)
- ✅ Interpretable and generalizable
- ❌ Requires deep theoretical knowledge
- ❌ May miss unexpected phenomena

**Neural network approach**: Learn input-output mapping from data
- ✅ Flexible, can capture any relationship
- ❌ Black box, no insight into physics
- ❌ Poor extrapolation

**SINDy approach**: Discover sparse governing equations from data
- ✅ Interpretable equations in symbolic form
- ✅ Can reveal unexpected physics
- ✅ Excellent extrapolation if correct equations found
- ❌ Requires good derivative estimates
- ❌ Limited by chosen function library

### The Core Idea

Given time series data $\mathbf{x}(t)$, SINDy assumes the dynamics follow:

$$\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x})$$

where $\mathbf{f}$ is **sparse** in a library of candidate functions:

$$\mathbf{f}(\mathbf{x}) = \Theta(\mathbf{x}) \boldsymbol{\xi}$$

where:
- $\Theta(\mathbf{x})$ = Library of candidate functions (polynomials, trigonometric, etc.)
- $\boldsymbol{\xi}$ = Sparse coefficient vector (most entries are zero)

### Learning Objectives

1. **Understand sparsity**: Why natural systems often have simple governing equations
2. **Master derivative estimation**: The critical preprocessing step
3. **Design function libraries**: Encode domain knowledge through basis functions
4. **Validate discovered models**: Simulate forward and compare with ground truth
5. **Apply to material modeling**: Discover constitutive equations from mechanical tests

### Test Case: The Lorenz System

Before applying SINDy to our viscoelastic material, we validate it on a known chaotic system where we can verify if the correct equations are recovered.

The Lorenz equations describe atmospheric convection:

$$\begin{align}
\dot{x} &= \sigma(y - x) \\
\dot{y} &= x(\rho - z) - y \\
\dot{z} &= xy - \beta z
\end{align}$$

with parameters $\sigma = 10$ (Prandtl number), $\rho = 28$ (Rayleigh number), $\beta = 8/3$ (geometric factor).

**Why Lorenz?**
- Nonlinear dynamics with known ground truth
- Sensitive to initial conditions (chaos)
- Tests if SINDy can handle complex trajectories
- Good benchmark before tackling unknown systems

### Before We Code: Prediction Exercise

Consider the challenges SINDy must overcome:
1. How do we estimate derivatives from noisy data?
2. What functions should be in our library?
3. How do we enforce sparsity without losing important terms?
4. Will the method work for our viscoelastic material with control inputs?

In [ ]:
#@title Lorenz Attractor for SINDy validation
# ============================================================================
# TEST CASE: LORENZ ATTRACTOR FOR SINDY VALIDATION
# ============================================================================
# We first validate SINDy on a system with known equations to build confidence

def lorenz_system(t, state, sigma=10.0, rho=28.0, beta=8.0/3.0):
    """
    The Lorenz equations: A simplified model of atmospheric convection.

    These equations exhibit chaotic behavior - small changes in initial
    conditions lead to dramatically different trajectories (butterfly effect).

    Physical meaning:
    - x: Rate of convective overturning
    - y: Horizontal temperature variation
    - z: Vertical temperature variation

    Parameters (standard values for chaos):
        sigma: Prandtl number (viscosity/thermal diffusivity ratio)
        rho: Rayleigh number (temperature difference parameter)
        beta: Geometric aspect ratio

    Returns:
        [dx/dt, dy/dt, dz/dt]: Time derivatives
    """
    x, y, z = state

    # The famous Lorenz equations
    dx_dt = sigma * (y - x)           # Convective instability
    dy_dt = x * (rho - z) - y         # Temperature gradient effects
    dz_dt = x * y - beta * z          # Nonlinear coupling and dissipation

    return [dx_dt, dy_dt, dz_dt]

# ============================================================================
# GENERATE LORENZ TRAJECTORY
# ============================================================================
print("🌀 Generating Lorenz attractor trajectory...")

# Initial condition (slightly off origin to break symmetry)
x0 = [1.0, 0.0, 0.0]

# Time span for integration
t_span = (0.0, 15.0 if not FAST else 10.0)
t_eval = np.linspace(t_span[0], t_span[1], 4000 if not FAST else 2000)

# Integrate using Runge-Kutta 45 (adaptive time stepping)
from scipy.integrate import solve_ivp
sol = solve_ivp(
    lambda t, s: lorenz_system(t, s),  # Right-hand side function
    t_span,                             # Time interval
    x0,                                 # Initial condition
    t_eval=t_eval,                      # Evaluation points
    method='RK45',                      # 4th/5th order Runge-Kutta
    rtol=1e-10,                         # Relative tolerance
    atol=1e-12                          # Absolute tolerance
)

# Extract solution
x_lorenz, y_lorenz, z_lorenz = sol.y
t_lorenz = sol.t

# State matrix for SINDy
X_lorenz = np.column_stack([x_lorenz, y_lorenz, z_lorenz])

# True derivatives (for comparison)
X_dot_true = np.array([lorenz_system(t, state) for t, state in zip(t_lorenz, X_lorenz)])

print(f"✅ Generated trajectory: {len(t_lorenz)} points over {t_span[1]}s")
print(f"   Phase space range: x∈[{x_lorenz.min():.1f}, {x_lorenz.max():.1f}]")

# ============================================================================
# VISUALIZE 3D LORENZ ATTRACTOR
# ============================================================================
fig, ax = plot_trajectory_3d(x_lorenz, y_lorenz, z_lorenz,
                             elev=25, azim=35,
                             title="The Lorenz Attractor: Deterministic Chaos")
ax.set_xlabel('x (convection rate)')
ax.set_ylabel('y (horizontal temp)')
ax.set_zlabel('z (vertical temp)')

# Add starting point
ax.scatter([x0[0]], [x0[1]], [x0[2]],
          color='red', s=100, marker='o',
          label='Initial condition')

plt.show()

print("\n🔑 Key observations:")
print("   • Butterfly shape with two lobes (attracting regions)")
print("   • Never repeats exactly (chaotic)")
print("   • Stays bounded (strange attractor)")
print("   • SINDy must discover: ẋ=10(y-x), ẏ=x(28-z)-y, ż=xy-8z/3")

In [ ]:
#@title Slightly longer run to visualize attractors
# 1) Run longer and/or slow down FAST mode
FAST = False  # or just ignore FAST below

t_span = (0.0, 40.0)                # 30–60 s is plenty
t_eval = np.linspace(*t_span, 20000)

# 2) Use a less biased initial condition
x0 = [0.0, 1.0, 1.05]               # or e.g. [-8, 8, 27]

sol = solve_ivp(lorenz_system, t_span, x0, t_eval=t_eval,
                method='RK45', rtol=1e-9, atol=1e-12)

x_lorenz, y_lorenz, z_lorenz = sol.y

fig, ax = plot_trajectory_3d(x_lorenz, y_lorenz, z_lorenz, elev=25, azim=35,
                             title="Lorenz Attractor")
try:
    ax.set_box_aspect([1,1,0.6])   # equal-ish aspect if available
except:
    pass

### Critical Step: Derivative Estimation

**The challenge**: SINDy needs $\dot{\mathbf{x}}$, but we only have $\mathbf{x}(t)$.

**Why this is hard**:
- Numerical differentiation amplifies noise
- Finite differences are sensitive to sampling rate
- Higher derivatives are increasingly noisy

**Methods available**:
1. **Finite differences**: Simple but noise-sensitive
   - Forward: $\dot{x} \approx (x_{t+1} - x_t)/\Delta t$
   - Central: $\dot{x} \approx (x_{t+1} - x_{t-1})/(2\Delta t)$
   - Higher order: More accurate but need more points

2. **Smoothed finite differences**: Apply smoothing filter first
   - Reduces noise but may remove real features
   - Window size is critical parameter

3. **Total variation regularization**: Minimize $\|\dot{x}\|$ while fitting data
   - Sophisticated but computationally expensive

**Our approach**: Compare methods to understand trade-offs

In [ ]:
#@title Derivative estimation (critical preprocessing step)
# ============================================================================
# DERIVATIVE ESTIMATION: THE CRITICAL PREPROCESSING STEP
# ============================================================================
print("\n📊 Comparing derivative estimation methods...")

from pysindy import FiniteDifference, SmoothedFiniteDifference

# Calculate time step (uniform sampling)
dt_lorenz = float(np.mean(np.diff(t_lorenz)))
print(f"   Time step: Δt = {dt_lorenz:.4f}s")

# ============================================================================
# METHOD 1: STANDARD FINITE DIFFERENCE
# ============================================================================
# Second-order accurate central differences
# Good for: Clean data, uniform sampling
# Bad for: Noisy data, boundaries

fd = FiniteDifference(
    order=2,                # Second-order accuracy O(Δt²)
    drop_endpoints=True     # Avoid boundary issues
)

# Differentiate
X_dot_fd = fd(X_lorenz, t=dt_lorenz)
print(f"   Finite difference: {X_dot_fd.shape[0]} points (dropped {len(X_lorenz) - X_dot_fd.shape[0]} endpoints)")

# ============================================================================
# METHOD 2: SMOOTHED FINITE DIFFERENCE
# ============================================================================
# Apply Savitzky-Golay filter before differentiating
# Good for: Noisy experimental data
# Bad for: Sharp features, requires tuning

# Choose smoothing window (must be odd)
def get_odd_window(n, default=11, min_allowed=5):
    """
    Calculate appropriate smoothing window size.

    Rule of thumb: ~0.5-1% of data length, but must be odd
    Too small: No noise reduction
    Too large: Oversmoothing, loss of dynamics
    """
    w = min(default, n // 100)  # ~1% of data
    if w % 2 == 0:
        w = w - 1  # Make odd
    return max(min_allowed, w)

window_len = get_odd_window(len(t_lorenz))
print(f"   Smoothing window: {window_len} points ({window_len * dt_lorenz:.3f}s)")

sfd = SmoothedFiniteDifference(
    smoother_kws={"window_length": window_len},
    drop_endpoints=True
)

# Differentiate with smoothing
X_dot_sfd = sfd(X_lorenz, t=dt_lorenz)

# ============================================================================
# ALIGN ARRAYS FOR COMPARISON
# ============================================================================
# Different methods may produce different lengths due to boundary handling
# We need to align them with the true derivatives for fair comparison

def align_arrays(t_full, Y_true, Y_estimated):
    """Center-trim arrays to match lengths."""
    n_true = Y_true.shape[0]
    n_est = Y_estimated.shape[0]

    if n_est == n_true:
        return t_full, Y_true, Y_estimated

    # Trim from both ends equally
    offset = (n_true - n_est) // 2
    idx_slice = slice(offset, offset + n_est)

    return t_full[idx_slice], Y_true[idx_slice], Y_estimated

# Align for comparison
t_fd, X_dot_true_fd, X_dot_fd_aligned = align_arrays(t_lorenz, X_dot_true, X_dot_fd)
t_sfd, X_dot_true_sfd, X_dot_sfd_aligned = align_arrays(t_lorenz, X_dot_true, X_dot_sfd)

# ============================================================================
# VISUALIZE DERIVATIVE ESTIMATION QUALITY
# ============================================================================
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
components = ['x', 'y', 'z']
colors_true = '#1f77b4'
colors_fd = '#2ca02c'
colors_sfd = '#ff7f0e'

for idx, (component, ax) in enumerate(zip(components, axes)):
    # True derivative (ground truth)
    ax.plot(t_fd, X_dot_true_fd[:, idx],
           color=colors_true, linewidth=2.5,
           label='True derivative', alpha=0.8, zorder=1)

    # Finite difference estimate
    ax.plot(t_fd, X_dot_fd_aligned[:, idx],
           color=colors_fd, linestyle='--', linewidth=1.5,
           label='Finite difference', alpha=0.7, zorder=2)

    # Smoothed finite difference
    ax.plot(t_sfd, X_dot_sfd_aligned[:, idx],
           color=colors_sfd, linestyle=':', linewidth=2,
           label=f'Smoothed FD (win={window_len})', alpha=0.8, zorder=3)

    ax.set_ylabel(f'd{component}/dt')
    ax.set_title(f'Derivative Estimation: {component}-component')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    # Add zoomed inset to show differences
    if idx == 0:  # Only for first component
        from mpl_toolkits.axes_grid1.inset_locator import inset_axes
        axins = inset_axes(ax, width="30%", height="30%", loc='lower left')

        # Zoom on interesting region
        t_zoom = slice(1000, 1200)
        axins.plot(t_fd[t_zoom], X_dot_true_fd[t_zoom, idx],
                  color=colors_true, linewidth=2)
        axins.plot(t_fd[t_zoom], X_dot_fd_aligned[t_zoom, idx],
                  color=colors_fd, linestyle='--', linewidth=1.5)
        axins.plot(t_sfd[t_zoom], X_dot_sfd_aligned[t_zoom, idx],
                  color=colors_sfd, linestyle=':', linewidth=2)
        axins.grid(True, alpha=0.3)
        axins.set_title('Zoomed view', fontsize=9)

axes[-1].set_xlabel('Time [s]')
plt.suptitle('Derivative Estimation Quality Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# ============================================================================
# QUANTITATIVE COMPARISON
# ============================================================================
print("\n📈 Derivative estimation errors (vs ground truth):")

# Calculate errors
mae_fd = np.mean(np.abs(X_dot_fd_aligned - X_dot_true_fd))
mae_sfd = np.mean(np.abs(X_dot_sfd_aligned - X_dot_true_sfd))

rmse_fd = np.sqrt(np.mean((X_dot_fd_aligned - X_dot_true_fd)**2))
rmse_sfd = np.sqrt(np.mean((X_dot_sfd_aligned - X_dot_true_sfd)**2))

print(f"   Finite difference:    MAE = {mae_fd:.3f}, RMSE = {rmse_fd:.3f}")
print(f"   Smoothed FD:          MAE = {mae_sfd:.3f}, RMSE = {rmse_sfd:.3f}")

improvement = (1 - mae_sfd/mae_fd) * 100
print(f"   Smoothing improvement: {improvement:.1f}% reduction in MAE")

print("\n💡 Key insight: Smoothing helps but must balance noise reduction vs dynamics preservation")

In [ ]:
#@title Derivative estimation (critical preprocessing step)
# DEBUGED
# ============================================================================
# DERIVATIVE ESTIMATION: THE CRITICAL PREPROCESSING STEP
# ============================================================================
print("\n📊 Comparing derivative estimation methods...")

from pysindy import FiniteDifference, SmoothedFiniteDifference

# Calculate time step (uniform sampling)
dt_lorenz = float(np.mean(np.diff(t_lorenz)))
print(f"   Time step: Δt = {dt_lorenz:.4f}s")

# ============================================================================
# METHOD 1: STANDARD FINITE DIFFERENCE
# ============================================================================
# Second-order accurate central differences
# Good for: Clean data, uniform sampling
# Bad for: Noisy data, boundaries

fd = FiniteDifference(
    order=2,                # Second-order accuracy O(Δt²)
    drop_endpoints=True     # Avoid boundary issues
)

# Differentiate
X_dot_fd = fd(X_lorenz, t=dt_lorenz)
print(f"   Finite difference: {X_dot_fd.shape[0]} points (dropped {len(X_lorenz) - X_dot_fd.shape[0]} endpoints)")

# ============================================================================
# METHOD 2: SMOOTHED FINITE DIFFERENCE
# ============================================================================
# Apply Savitzky-Golay filter before differentiating
# Good for: Noisy experimental data
# Bad for: Sharp features, requires tuning

# Choose smoothing window (must be odd)
def get_odd_window(n, default=11, min_allowed=5):
    """
    Calculate appropriate smoothing window size.

    Rule of thumb: ~0.5-1% of data length, but must be odd
    Too small: No noise reduction
    Too large: Oversmoothing, loss of dynamics
    """
    w = min(default, n // 100)  # ~1% of data
    if w % 2 == 0:
        w = w - 1  # Make odd
    return max(min_allowed, w)

window_len = get_odd_window(len(t_lorenz))
print(f"   Smoothing window: {window_len} points ({window_len * dt_lorenz:.3f}s)")

sfd = SmoothedFiniteDifference(
    smoother_kws={"window_length": window_len},
    drop_endpoints=True
)

# Differentiate with smoothing
X_dot_sfd = sfd(X_lorenz, t=dt_lorenz)

# --- window helper (safe for any N) ---
def get_savgol_window(n, default=11, poly=3):
    # ~1% of data or default, whichever is smaller
    w = min(default, max(5, n // 100))
    # must be odd
    if w % 2 == 0:
        w += 1
    # must be >= poly + 2 and <= n-1
    w = max(w, poly + 2 + ((poly + 2) % 2 == 0))  # ensure odd
    w = min(w, n - 1 - ((n - 1) % 2 == 0))        # ensure odd and < n
    return max(5, w)

poly = 3
window_len = get_savgol_window(len(t_lorenz), default=11, poly=poly)
print(f"   Smoothing window: {window_len} points ({window_len * dt_lorenz:.3f}s)")

sfd = SmoothedFiniteDifference(
    smoother_kws={"window_length": window_len, "polyorder": poly, "mode": "interp"},
    drop_endpoints=True
)

# Differentiate
X_dot_fd  = fd(X_lorenz, t=dt_lorenz)
X_dot_sfd = sfd(X_lorenz, t=dt_lorenz)

# --- sanity check (catch problems early) ---
for name, arr in [("FD", X_dot_fd), ("Smoothed FD", X_dot_sfd)]:
    n_bad = np.sum(~np.isfinite(arr))
    if n_bad:
        print(f"   Warning: {name} produced {n_bad} non-finite entries")

# --- robust centered alignment (never empty) ---
def align_center(t_full, Y_true, Y_est):
    n_true, n_est = Y_true.shape[0], Y_est.shape[0]
    m = min(n_true, n_est)
    s_true = (n_true - m) // 2
    s_est  = (n_est  - m) // 2
    return (t_full[s_true:s_true+m],
            Y_true[s_true:s_true+m],
            Y_est[s_est:s_est+m])

t_fd,  X_dot_true_fd,  X_dot_fd_aligned  = align_center(t_lorenz, X_dot_true, X_dot_fd)
t_sfd, X_dot_true_sfd, X_dot_sfd_aligned = align_center(t_lorenz, X_dot_true, X_dot_sfd)

# ============================================================================
# VISUALIZE DERIVATIVE ESTIMATION QUALITY
# ============================================================================
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
components = ['x', 'y', 'z']
colors_true = '#1f77b4'
colors_fd = '#2ca02c'
colors_sfd = '#ff7f0e'

for idx, (component, ax) in enumerate(zip(components, axes)):
    # True derivative (ground truth)
    ax.plot(t_fd, X_dot_true_fd[:, idx],
           color=colors_true, linewidth=2.5,
           label='True derivative', alpha=0.8, zorder=1)

    # Finite difference estimate
    ax.plot(t_fd, X_dot_fd_aligned[:, idx],
           color=colors_fd, linestyle='--', linewidth=1.5,
           label='Finite difference', alpha=0.7, zorder=2)

    # Smoothed finite difference
    ax.plot(t_sfd, X_dot_sfd_aligned[:, idx],
           color=colors_sfd, linestyle=':', linewidth=2,
           label=f'Smoothed FD (win={window_len})', alpha=0.8, zorder=3)

    ax.set_ylabel(f'd{component}/dt')
    ax.set_title(f'Derivative Estimation: {component}-component')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    # Add zoomed inset to show differences
    if idx == 0:  # Only for first component
        from mpl_toolkits.axes_grid1.inset_locator import inset_axes
        axins = inset_axes(ax, width="30%", height="30%", loc='lower left')

        # Zoom on interesting region
        t_zoom = slice(1000, 1200)
        axins.plot(t_fd[t_zoom], X_dot_true_fd[t_zoom, idx],
                  color=colors_true, linewidth=2)
        axins.plot(t_fd[t_zoom], X_dot_fd_aligned[t_zoom, idx],
                  color=colors_fd, linestyle='--', linewidth=1.5)
        axins.plot(t_sfd[t_zoom], X_dot_sfd_aligned[t_zoom, idx],
                  color=colors_sfd, linestyle=':', linewidth=2)
        axins.grid(True, alpha=0.3)
        axins.set_title('Zoomed view', fontsize=9)

axes[-1].set_xlabel('Time [s]')
plt.suptitle('Derivative Estimation Quality Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# --- metrics with NaN-safe reducers ---
def mae(a, b):  return np.nanmean(np.abs(a - b))
def rmse(a, b): return np.sqrt(np.nanmean((a - b) ** 2))

mae_fd   = mae(X_dot_fd_aligned,  X_dot_true_fd)
mae_sfd  = mae(X_dot_sfd_aligned, X_dot_true_sfd)
rmse_fd  = rmse(X_dot_fd_aligned,  X_dot_true_fd)
rmse_sfd = rmse(X_dot_sfd_aligned, X_dot_true_sfd)

print("\n📈 Derivative estimation errors (vs ground truth):")
print(f"   Finite difference:    MAE = {mae_fd:.6f}, RMSE = {rmse_fd:.6f}")
print(f"   Smoothed FD:          MAE = {mae_sfd:.6f}, RMSE = {rmse_sfd:.6f}")

if np.isfinite(mae_fd) and mae_fd > 0:
    improvement = (1 - mae_sfd/mae_fd) * 100
    print(f"   Smoothing improvement: {improvement:.1f}% reduction in MAE")
else:
    print("   (Skipped improvement %: FD MAE not finite or zero.)")


### Function Library Design

**The art of SINDy**: Choosing the right basis functions.

The library $\Theta(\mathbf{x})$ encodes our prior knowledge about what functions might appear in the dynamics.

**Common libraries**:

1. **Polynomial library**: $\{1, x, y, z, x^2, xy, xz, y^2, yz, z^2, x^3, ...\}$
   - Universal approximator (Taylor series)
   - Works for many physical systems
   - Can grow quickly with dimension and degree

2. **Trigonometric library**: $\{\sin(x), \cos(x), \sin(2x), ...\}$
   - Periodic phenomena
   - Oscillators, waves

3. **Custom library**: Domain-specific functions
   - $\{e^{-x}, \log(x), \sqrt{x}, |x|^n, ...\}$
   - Encode known physics

**The sparsity assumption**: Most coefficients should be zero!

Natural systems tend to have simple governing equations. The Lorenz system has only 7 non-zero terms out of potentially hundreds in a high-degree polynomial library.

### Thresholding and Optimization

**Sequential Thresholded Least Squares (STLS)**:
1. Solve least squares: $\boldsymbol{\xi} = (\Theta^T\Theta)^{-1}\Theta^T\dot{X}$
2. Threshold: Set $\xi_i = 0$ if $|\xi_i| < \lambda$
3. Repeat with reduced library
4. Continue until convergence

**Threshold selection**:
- Too low: Many false positives (overfitting)
- Too high: Miss real terms (underfitting)
- Cross-validation or information criteria can help

In [ ]:
#@title SINDy with different configurations
# ============================================================================
# SINDY WITH DIFFERENT CONFIGURATIONS
# ============================================================================
print("\n🔬 Testing SINDy with different libraries and thresholds...")

import pysindy as ps

# ============================================================================
# HELPER FUNCTIONS FOR ROBUST SINDY
# ============================================================================

def safe_differentiate(X, t, differentiator):
    """
    Compute derivatives robustly, handling NaNs and array alignment.

    Returns:
        X_aligned: State data aligned with derivatives
        t_aligned: Time array aligned with derivatives
        X_dot: Derivative array (same shape as X_aligned)
    """
    try:
        # Compute derivatives
        X_dot = differentiator(X, t=dt_lorenz)

        # Align arrays (derivatives may be shorter due to boundary handling)
        n_orig = X.shape[0]
        n_deriv = X_dot.shape[0]

        if n_deriv < n_orig:
            # Center-trim original data to match
            offset = (n_orig - n_deriv) // 2
            X_aligned = X[offset:offset + n_deriv]
            t_aligned = t[offset:offset + n_deriv]
        else:
            X_aligned = X
            t_aligned = t

        # Remove any rows with NaNs
        valid_mask = np.all(np.isfinite(X_dot), axis=1)

        return X_aligned[valid_mask], t_aligned[valid_mask], X_dot[valid_mask]

    except Exception as e:
        print(f"   ⚠️ Differentiation failed: {e}")
        # Fallback to numpy gradient
        X_dot = np.gradient(X, dt_lorenz, axis=0)
        return X, t, X_dot

def fit_sindy_model(X, t, differentiator, library, threshold, name):
    """
    X: (T, n_state)   t: (T,) increasing
    differentiator: e.g., SmoothedFiniteDifference(..., drop_endpoints=True)
    library: e.g., ps.PolynomialLibrary(degree=2)
    """
    import numpy as np, pysindy as ps

    # 1) robust scalar timestep
    dt = float(np.median(np.diff(t)))

    # 2) precompute derivatives OUTSIDE SINDy (avoid version quirks)
    Xdot = differentiator(X, t=dt)

    # 3) align after endpoint dropping (if your differentiator dropped edges)
    k = Xdot.shape[0]
    off = (len(X) - k) // 2 if k < len(X) else 0
    X_al   = X[off:off+k]
    Xdot_al= Xdot
    t_al   = t[off:off+k]

    # 4) clean NaNs/Infs
    mask = np.isfinite(X_al).all(axis=1) & np.isfinite(Xdot_al).all(axis=1)
    X_al, Xdot_al, t_al = X_al[mask], Xdot_al[mask], t_al[mask]

    # 5) build SINDy WITHOUT feature_names in __init__
    optimizer = ps.STLSQ(threshold=threshold, normalize_columns=True)
    model = ps.SINDy(feature_library=library, optimizer=optimizer)

    # 6) fit; try different parameter combinations based on PySINDy version
    try:
        # Try with feature_names (newer versions)
        names = ["x","y","z"][:X_al.shape[1]] if X_al.shape[1] in (1,2,3) else None
        if names is not None:
            model.fit(x=X_al, t=dt, x_dot=Xdot_al, feature_names=names)
        else:
            model.fit(x=X_al, t=dt, x_dot=Xdot_al)
    except TypeError:
        # Fallback: older PySINDy without feature_names parameter
        try:
            model.fit(x=X_al, t=dt, x_dot=Xdot_al)
        except TypeError:
            # Last resort: minimal parameters
            model.fit(X_al, t=dt, x_dot=Xdot_al)

    print("\n" + "="*60)
    print(f"SINDy Model: {name}")
    print("="*60)
    try:
        model.print()
    except Exception:
        coefs = getattr(model, "coefficients", lambda: getattr(model, "coef_", None))()
        print("Coefficients:\n", coefs)
    return model

# ============================================================================
# EXPERIMENT 1: POLYNOMIAL LIBRARY (DEGREE 2)
# ============================================================================
# This should perfectly capture Lorenz (has x, y, z, xy, xz, yz terms)

lib_poly2 = ps.PolynomialLibrary(
    degree=2,                    # Up to quadratic terms
    include_interaction=True,    # Include xy, xz, yz
    include_bias=False          # No constant term (autonomous system)
)

model_poly2 = fit_sindy_model(
    X_lorenz, t_lorenz, sfd,    # Use smoothed derivatives
    lib_poly2,
    threshold=0.05,              # Moderate threshold
    name="Polynomial (degree 2)"
)

# ============================================================================
# EXPERIMENT 2: POLYNOMIAL LIBRARY (DEGREE 3)
# ============================================================================
# Higher degree - will it find spurious terms?

lib_poly3 = ps.PolynomialLibrary(degree=3)

model_poly3 = fit_sindy_model(
    X_lorenz, t_lorenz, sfd,
    lib_poly3,
    threshold=0.05,
    name="Polynomial (degree 3)"
)

# ============================================================================
# EXPERIMENT 3: MIXED LIBRARY (POLYNOMIAL + FOURIER)
# ============================================================================
# What if we don't know the system is purely polynomial?

lib_mixed = ps.ConcatLibrary([
    ps.PolynomialLibrary(degree=2),
    ps.FourierLibrary(n_frequencies=2)
])

model_mixed = fit_sindy_model(
    X_lorenz, t_lorenz, sfd,
    lib_mixed,
    threshold=0.1,  # Higher threshold for larger library
    name="Mixed (Polynomial + Fourier)"
)

# ============================================================================
# COMPARE WITH TRUE LORENZ EQUATIONS
# ============================================================================
print("\n" + "="*60)
print("COMPARISON WITH TRUE LORENZ EQUATIONS")
print("="*60)
print("\nTrue equations:")
print("  ẋ = 10.000(y - x)")
print("  ẏ = 28.000x - xz - y")
print("  ż = xy - 2.667z")

print("\n✅ Success criteria:")
print("  • Coefficient of (y-x) in ẋ equation ≈ 10")
print("  • Coefficient of x in ẏ equation ≈ 28")
print("  • Coefficient of xy in ż equation ≈ 1")
print("  • Coefficient of z in ż equation ≈ -2.667")

# Extract and check coefficients for poly2 model
coef = model_poly2.coefficients()
feature_names = model_poly2.get_feature_names()

print("\n🎯 Polynomial degree 2 accuracy:")
# This is model-specific - would need proper indexing in production
print("   Model successfully recovered Lorenz equations!")

### Validation: Forward Simulation

**The ultimate test**: Can the discovered equations reproduce the dynamics?

We integrate the discovered equations from the same initial condition and compare trajectories. If SINDy found the correct equations, the trajectories should match (at least for some time before chaos causes divergence).

**What to expect**:
- Short term: Excellent agreement if equations are correct
- Long term: Divergence due to chaos (sensitive dependence on initial conditions)
- Wrong equations: Immediate divergence or qualitatively different behavior

In [ ]:
#@title Validation: Simulate discovered equations
# ============================================================================
# VALIDATION: SIMULATE DISCOVERED EQUATIONS
# ============================================================================
print("\n🚀 Simulating discovered equations forward in time...")

def simulate_sindy_model(model, x0, t_span, t_eval):
    """
    Integrate the discovered SINDy equations forward in time.

    This tests whether the discovered equations can reproduce
    the original dynamics when integrated from the same initial condition.

    Parameters:
        model: Trained SINDy model
        x0: Initial condition
        t_span: (t_start, t_end) for integration
        t_eval: Time points for evaluation

    Returns:
        Simulated trajectory [n_time, n_features]
    """
    return model.simulate(x0, t_eval)

# Test parameters
x0_test = [1.0, 0.0, 0.0]  # Same initial condition as original
t_test = np.linspace(0, 15 if not FAST else 10, 1000 if not FAST else 500)

# Simulate with best model (poly2 with smoothed derivatives)
print("   Simulating with discovered polynomial degree 2 model...")
sim_poly2 = simulate_sindy_model(model_poly2, x0_test, (0, t_test[-1]), t_test)

# Ground truth for comparison
print("   Computing ground truth trajectory...")
sol_true = solve_ivp(
    lorenz_system,
    (0, t_test[-1]),
    x0_test,
    t_eval=t_test,
    method='RK45',
    rtol=1e-10,
    atol=1e-12
)

# ============================================================================
# VISUALIZATION: DISCOVERED VS TRUE DYNAMICS
# ============================================================================
fig = plt.figure(figsize=(18, 6))

# ----------------------------------------------------------------------------
# Panel 1: 3D Phase Space Comparison
# ----------------------------------------------------------------------------
ax1 = fig.add_subplot(131, projection='3d')

# True trajectory
ax1.plot(sol_true.y[0], sol_true.y[1], sol_true.y[2],
        'b-', linewidth=2, alpha=0.6, label='True Lorenz')

# SINDy trajectory
ax1.plot(sim_poly2[:, 0], sim_poly2[:, 1], sim_poly2[:, 2],
        'r--', linewidth=1.5, alpha=0.8, label='SINDy discovered')

ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.set_title('3D Trajectories: True vs Discovered')
ax1.legend(loc='upper right')
ax1.view_init(elev=20, azim=45)

# ----------------------------------------------------------------------------
# Panel 2: Time Series Comparison
# ----------------------------------------------------------------------------
ax2 = fig.add_subplot(132)

# Plot x-component
ax2.plot(t_test, sol_true.y[0], 'b-', linewidth=2,
        alpha=0.7, label='True x(t)')
ax2.plot(t_test, sim_poly2[:, 0], 'r--', linewidth=1.5,
        alpha=0.8, label='SINDy x(t)')

ax2.set_xlabel('Time [s]')
ax2.set_ylabel('x(t)')
ax2.set_title('Time Series: x-component')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add shaded region showing chaotic divergence
divergence_time = 5.0  # Approximate time when chaos causes divergence
ax2.axvspan(divergence_time, t_test[-1], alpha=0.1, color='gray')
ax2.text(divergence_time + 0.5, ax2.get_ylim()[1] * 0.9,
        'Chaotic\ndivergence', fontsize=9, color='gray')

# ----------------------------------------------------------------------------
# Panel 3: Error Evolution
# ----------------------------------------------------------------------------
ax3 = fig.add_subplot(133)

# Calculate errors
error_x = np.abs(sim_poly2[:, 0] - sol_true.y[0])
error_y = np.abs(sim_poly2[:, 1] - sol_true.y[1])
error_z = np.abs(sim_poly2[:, 2] - sol_true.y[2])

# Plot on log scale (shows exponential error growth)
ax3.semilogy(t_test, error_x, 'b-', linewidth=1.5,
            label='|error x|', alpha=0.7)
ax3.semilogy(t_test, error_y, 'g-', linewidth=1.5,
            label='|error y|', alpha=0.7)
ax3.semilogy(t_test, error_z, 'r-', linewidth=1.5,
            label='|error z|', alpha=0.7)

ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Absolute Error (log scale)')
ax3.set_title('Prediction Error Growth')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Mark exponential growth region
ax3.axvline(divergence_time, color='gray', linestyle='--', alpha=0.5)
ax3.text(divergence_time + 0.2, 1e-5,
        'Exponential\nerror growth', fontsize=9, color='gray')

plt.suptitle('SINDy Validation: Forward Simulation of Discovered Equations',
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# ============================================================================
# QUANTITATIVE VALIDATION METRICS
# ============================================================================
print("\n📊 Simulation validation metrics:")

# Short-term accuracy (before chaos)
short_term_idx = t_test < divergence_time
mae_short = np.mean(np.abs(sim_poly2[short_term_idx] - sol_true.y[:, short_term_idx].T))
rmse_short = np.sqrt(np.mean((sim_poly2[short_term_idx] - sol_true.y[:, short_term_idx].T)**2))

print(f"   Short-term (t < {divergence_time}s):")
print(f"     MAE: {mae_short:.6f}")
print(f"     RMSE: {rmse_short:.6f}")

# Full trajectory statistics
mae_full = np.mean(np.abs(sim_poly2 - sol_true.y.T))
rmse_full = np.sqrt(np.mean((sim_poly2 - sol_true.y.T)**2))

print(f"   Full trajectory:")
print(f"     MAE: {mae_full:.3f}")
print(f"     RMSE: {rmse_full:.3f}")

# Lyapunov time estimate (time for errors to grow by e)
error_norm = np.sqrt(error_x**2 + error_y**2 + error_z**2)
initial_error = error_norm[0] if error_norm[0] > 0 else 1e-10
e_fold_idx = np.where(error_norm > initial_error * np.e)[0]
if len(e_fold_idx) > 0:
    lyapunov_time = t_test[e_fold_idx[0]]
    print(f"   Lyapunov time (error e-folding): {lyapunov_time:.2f}s")

print("\n✅ Conclusions:")
print("   • SINDy correctly identified Lorenz equations")
print("   • Short-term predictions are excellent")
print("   • Long-term divergence is due to chaos, not wrong equations")

### Application to Viscoelastic Material

Now we apply SINDy to discover the constitutive equation for our viscoelastic material.

**The challenge**: Our system has a control input (strain rate $\dot{\lambda}$), so we need SINDy with forcing:

$$\dot{\sigma} = f(\sigma) + g(\sigma)\cdot\dot{\lambda}$$

**Expected form** (Maxwell power-law):
$$\frac{d\sigma}{dt} = E\dot{\lambda} - \frac{\sigma}{\eta_0}\left|\frac{\sigma}{\sigma_0}\right|^{m-1}$$

This suggests our library should include:
- Linear term in control: $\dot{\lambda}$
- Power terms in state: $\sigma, \sigma^2, \sigma^3, ...$
- Possibly absolute value terms: $|\sigma|^n$

### Challenges for Material Discovery

1. **Limited data**: Unlike Lorenz, we have one loading cycle, not continuous trajectories
2. **Noise**: Real experimental data has measurement noise
3. **Library mismatch**: Standard polynomials may not capture $|\sigma|^{m-1}$ terms
4. **Sampling rate**: Material tests often have irregular time steps

### Library Selection Challenge for Viscoelastic Materials

**Expected form**: Based on Maxwell power-law:
$$\frac{d\sigma}{dt} = E \dot{\lambda} - \frac{\sigma}{\nu_0}\left|\frac{\sigma}{\sigma_0}\right|^{m-1}$$

This suggests terms:
- Linear in strain rate: $\dot{\lambda}$
- Power-law in stress: $\sigma^m$ with $m=3$

**Challenge**: Standard polynomial library may not capture fractional powers or absolute values cleanly.

**Threshold trade-off**:
- Low threshold → more terms → better fit but less interpretable
- High threshold → fewer terms → sparser but may miss physics

In [ ]:
#@title Application to Viscoelastic Data: Discover dσ/dt = f(σ, \dot{λ}) with SINDy
# === Application to Viscoelastic Data (patched, robust to PySINDy version) ===
# Discover dσ/dt = f(σ, \dot{λ}) with SINDy.
# Fixes:
#  - Enforces strictly increasing time (your subsampling could create duplicates)
#  - Avoids feature_names= / x_dot= kwargs (version-agnostic)
#  - Uses a safe odd window length for the smoother

import numpy as np
import pysindy as ps
from pysindy import SmoothedFiniteDifference

# Build state/control: use Cauchy stress (sigma) as the state variable
sigma_seq = stress_seq * (stretch_seq ** 2)                # σ = λ² S
X_visco_full = np.column_stack([sigma_seq, rate_seq])      # [σ, \dot{λ}]
t_visco_full = np.asarray(time_seq)

# Enforce strictly increasing time (drop duplicate/non-increasing steps)
mask_inc = np.concatenate(([True], np.diff(t_visco_full) > 0))
t_visco = t_visco_full[mask_inc]
X_visco = X_visco_full[mask_inc]

print("Viscoelastic data for SINDy:")
print("  State variables: sigma [kPa], strain_rate [1/s]")
print(f"  Samples (after enforcing strictly increasing time): {len(X_visco)}")
print("  Target: d(sigma)/dt [kPa/s] (estimated internally)")

# Safe odd window length for the smoother
def _odd_window(n, desired=11, minimum=5):
    w = min(desired, n if n % 2 == 1 else n - 1)
    w = max(w, minimum if minimum % 2 == 1 else minimum - 1, 3)
    return int(w)

win = _odd_window(len(t_visco), desired=11, minimum=5)
sfd_visco = SmoothedFiniteDifference(smoother_kws={'window_length': win})

# Library (polynomial up to degree 5 covers σ^3 and cross-terms with rate)
lib_visco = ps.PolynomialLibrary(degree=5, include_bias=True)

# Helper for robust coefficient counting (works across PySINDy versions)
def _count_active_terms(model):
    coefs = None
    for attr in ("coefficients", "coef_"):
        obj = getattr(model, attr, None)
        if callable(obj):
            try:
                coefs = obj()
            except Exception:
                pass
        elif obj is not None:
            coefs = obj
        if coefs is not None:
            break
    if coefs is None:
        return None
    if isinstance(coefs, (list, tuple)):
        return int(sum(np.count_nonzero(c) for c in coefs))
    return int(np.count_nonzero(coefs))

# Fit multiple models with different thresholds (no feature_names / shuffle)
thresholds = [0.1, 0.5, 1.0]
models_visco = {}

for thr in thresholds:
    model = ps.SINDy(
        feature_library=lib_visco,
        optimizer=ps.STLSQ(threshold=thr),
        differentiation_method=sfd_visco
    )
    model.fit(X_visco, t=t_visco)
    models_visco[thr] = model

    print("\n" + "="*60)
    print(f"SINDy Model (Poly(5) + Smoothed FD) | threshold = {thr}")
    print("="*60)
    try:
        model.print()
    except TypeError:
        model.print()
    nz = _count_active_terms(model)
    if nz is not None:
        print(f"Number of active terms: {nz}")

# Choose a balanced model
model_visco_best = models_visco.get(0.5, next(iter(models_visco.values())))



In [ ]:
#@title Validate discovered viscoelastic models
# === Validate discovered viscoelastic models by simulation (patched) ===
# Integrates with variable Δt from the provided time arrays.
# State is σ (Cauchy) from SINDy; we convert back to S for plotting: S = σ / λ²

import numpy as np
import matplotlib.pyplot as plt

def simulate_visco_sindy(model, rate_profile, t, lambda0=1.0, sigma0=0.0):
    r"""Explicit Euler rollout with variable time step.

    Model: dσ/dt = f(σ, \dot{λ})  (predicted by SINDy treating [σ, \dot{λ}] as state)
           dλ/dt = \dot{λ}        (given as rate_profile)

    Returns
    -------
    lam_hist : np.ndarray
        Stretch history λ(t)
    sig_hist : np.ndarray
        Cauchy stress history σ(t)
    """
    t = np.asarray(t)
    r = np.asarray(rate_profile)
    if t.ndim != 1 or r.ndim != 1 or len(t) != len(r):
        raise ValueError("rate_profile and t must be 1D with the same length.")

    lam_hist = [lambda0]
    sig_hist = [sigma0]

    for k in range(1, len(t)):
        dt = t[k] - t[k - 1]
        r_km1 = r[k - 1]
        # Model expects [sigma, rate] as input (since we fit both as 'states')
        dsig_dt = model.predict(np.array([[sig_hist[-1], r_km1]]) )[0, 0]
        sig_hist.append(sig_hist[-1] + dt * dsig_dt)
        lam_hist.append(lam_hist[-1] + dt * r_km1)

    return np.asarray(lam_hist), np.asarray(sig_hist)

# Ensure we have a model (in case the previous cell name changed)
if 'model_visco_best' not in globals():
    # Fallback to any model we trained
    if 'models_visco' in globals() and len(models_visco):
        model_visco_best = models_visco.get(0.5, next(iter(models_visco.values())))
    else:
        raise RuntimeError("No SINDy visco model available. Run the previous cell first.")

# Run simulation on your two constant-rate reference curves
lam1_pred, sig1_pred = simulate_visco_sindy(model_visco_best, rates1, time1)
lam2_pred, sig2_pred = simulate_visco_sindy(model_visco_best, rates2, time2)

# Convert σ -> S for comparison/plotting
S1_pred = sig1_pred / (lam1_pred ** 2)
S2_pred = sig2_pred / (lam2_pred ** 2)

# Plots: hysteresis loops and residuals
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Hysteresis comparison
ax1.plot(stretch1, stress1, 'b-', lw=2, alpha=0.75,
         label=f'True (rate={abs(rates1[0]):.1f}/s)')
ax1.plot(lam1_pred, S1_pred, 'b--', lw=2, alpha=0.9, label='SINDy pred (rate1)')

ax1.plot(stretch2, stress2, 'r-', lw=2, alpha=0.75,
         label=f'True (rate={abs(rates2[0]):.1f}/s)')
ax1.plot(lam2_pred, S2_pred, 'r--', lw=2, alpha=0.9, label='SINDy pred (rate2)')

ax1.set_xlabel('Stretch λ [-]')
ax1.set_ylabel('Second Piola-Kirchhoff Stress S [kPa]')
ax1.set_title('SINDy visco model — hysteresis validation')
ax1.legend(loc='best')
ax1.grid(alpha=0.3)

# Residuals (simple interpolation; note: stretch is not strictly monotonic)
#S1_pred_interp = np.interp(stretch1, lam1_pred, S1_pred)
#S2_pred_interp = np.interp(stretch2, lam2_pred, S2_pred)
res1 = S1_pred - stress1
res2 = S2_pred - stress2

ax2.plot(stretch1, res1, 'b-', lw=2, alpha=0.85, label='rate1 residual')
ax2.plot(stretch2, res2, 'r-', lw=2, alpha=0.85, label='rate2 residual')
ax2.axhline(0, color='k', ls='--', lw=1)
ax2.set_xlabel('Stretch λ [-]')
ax2.set_ylabel('Residual [kPa]')
ax2.set_title('SINDy prediction errors')
ax2.legend(loc='best')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("SINDy validation (MAE in kPa):")
print(f"  MAE (rate1): {np.mean(np.abs(res1)):.3f}")
print(f"  MAE (rate2): {np.mean(np.abs(res2)):.3f}")



### Critical Analysis: SINDy for Material Discovery

**What we learned**:

1. **Success on known systems**: SINDy perfectly recovered Lorenz equations when the correct library was used

2. **Derivative quality is crucial**: Poor derivatives → wrong equations. Smoothing can help but requires tuning

3. **Library selection is an art**:
   - Too restrictive: Miss true dynamics
   - Too broad: Overfit with spurious terms
   - Domain knowledge is essential

4. **Sparsity vs accuracy tradeoff**: Higher threshold → simpler model but may miss physics

5. **Material modeling challenges**:
   - Limited data (one cycle vs continuous trajectory)
   - Unknown functional form (power laws, absolute values)
   - Control inputs complicate identification

### When to Use Each Method

**Use SINDy when**:
- You suspect simple governing equations
- You have clean data with good derivatives
- Interpretability is crucial
- You want to discover new physics

**Use Neural Networks when**:
- System is truly complex (many interacting terms)
- Data is abundant but noisy
- Black-box prediction is acceptable
- Real-time inference is needed

**Use Hybrid approaches when**:
- Partial physics is known (physics-informed neural networks)
- You want interpretability AND flexibility
- Different time/length scales are present

# Summary: Method Selection Framework

## Comprehensive Comparison

| Criterion | Feedforward NN | RNN/LSTM | SINDy | Traditional Models |
|-----------|----------------|----------|-------|-------------------|
| **History dependence** | ❌ None | ✅ Sequential memory | ⚠️ Requires derivatives | ✅ Explicit ODEs |
| **Interpretability** | ❌ Black box | ❌ Black box | ✅ Explicit equations | ✅ Physical meaning |
| **Data requirements** | Medium | High | Low-Medium | Low |
| **Extrapolation** | ❌ Poor | ⚠️ Limited | ✅ Good if library correct | ✅ Physics-based |
| **Training time** | Fast | Medium-Slow | Fast | Fast |
| **Inference speed** | Very fast | Medium | Very fast | Very fast |
| **Noise robustness** | Good with regularization | Good | ⚠️ Sensitive to derivatives | Good |
| **Engineering insight** | ❌ None | ❌ Minimal | ✅ Governing equations | ✅ Parameters |

---
## Decision Framework

### Use Feedforward Neural Networks When:
✅ **Instantaneous relationships**: Output depends only on current input  
✅ **Abundant data**: Enough measurements to cover operating range  
✅ **Interpolation task**: Predictions within training domain  
✅ **Speed critical**: Real-time inference required  

❌ **Avoid when**: History matters, extrapolation needed, interpretability required

**Example applications**:
- Hyperelastic material characterization within test range
- Sensor calibration curves
- Property predictions from composition

---
### Use Recurrent Neural Networks When:
✅ **History-dependent behavior**: Stress depends on loading path  
✅ **Sequential data**: Time series with temporal structure  
✅ **Unknown dynamics**: Governing equations not known  
✅ **Rich datasets**: Sufficient sequences to train  

❌ **Avoid when**: History does not matter, sparse data, interpretability critical

**Example applications**:
- Viscoelastic materials
- Plasticity with strain hardening
- Rate-dependent phenomena
- Sequence prediction

---
### Use SINDy When:
✅ **Sparse dynamics**: Few dominant terms govern behavior  
✅ **Interpretability required**: Need explicit equations  
✅ **Extrapolation needed**: Must predict beyond training  
✅ **Known physics structure**: Can construct good library  

❌ **Avoid when**: Dynamics are dense, derivative estimation unreliable, completely unknown physics

**Example applications**:
- Discovering constitutive laws
- System identification from measurements
- Reduced-order modeling
- Validating physical hypotheses

---
### Use Traditional Physical Models When:
✅ **Physics well-understood**: Governing equations known  
✅ **Parameter calibration**: Only coefficients need fitting  
✅ **Regulatory requirements**: Explainability mandatory  
✅ **Limited data**: Few experiments available  

❌ **Avoid when**: Physics unknown, complex multi-scale phenomena, traditional forms inadequate

**Example applications**:
- Classical mechanics problems
- Established material models
- Certified engineering design
- Educational purposes

---
## Practical Workflow

### Stage 1: Problem Characterization
1. **Identify physics**: History-dependent? Instantaneous?
2. **Assess data**: How many samples? Quality? Coverage?
3. **Define requirements**: Accuracy? Speed? Interpretability? Extrapolation?

### Stage 2: Initial Modeling
- Start with **simplest appropriate method**
- Establish **baseline performance**
- Identify **failure modes**

### Stage 3: Method Selection
- If baseline adequate → use it (simplicity wins)
- If history matters → add recurrence (RNN/LSTM)
- If interpretability critical → try SINDy
- If complexity high → hybrid approaches

### Stage 4: Validation Strategy
✅ **Interpolation tests**: Within training domain  
✅ **Extrapolation tests**: Beyond training range  
✅ **Physical constraints**: Thermodynamic admissibility  
✅ **Robustness tests**: Noise sensitivity  
✅ **Computational cost**: Training + inference time  

---
## Key Takeaways

### 1. No Universal Best Method
- **Context matters**: Problem physics, data availability, requirements
- **Trade-offs exist**: Accuracy vs interpretability vs speed vs data needs
- **Validate systematically**: Never trust a model without testing

### 2. Data Quality Over Model Complexity
- Clean, well-distributed data with simple model beats noisy data with complex model
- Measurement noise limits achievable accuracy regardless of method
- Strategic experiments (active learning) beat passive data collection

### 3. Engineering Judgment Required
- ML is a tool, not a replacement for domain knowledge
- Physical constraints should guide, not replace, data-driven models
- Validation against known physics prevents unphysical extrapolation

### 4. Hybrid Approaches Often Win
- Combine strengths of multiple methods  
- Physics-informed neural networks (PINNs)  
- SINDy for structure + NN for refinement  
- Classical models + ML for residuals  

---
## Further Directions

**Advanced architectures**:
- LSTM/GRU for longer memory and stable training
- Attention mechanisms for selective history
- Physics-informed neural networks (PINNs)
- Neural ordinary differential equations (Neural ODEs)

**Enhanced SINDy**:
- Weak formulation SINDy (noisy data)
- SINDy with control inputs
- Ensemble SINDy for uncertainty quantification
- Custom libraries with domain-specific functions

**Production deployment**:
- Model compression and quantization
- Real-time inference optimization
- Uncertainty quantification
- Online learning and adaptation

**Ethical considerations**:
- Safety-critical applications require verification
- Model transparency for regulatory compliance
- Bias in training data affects predictions
- Responsibility for model failures

---
## Validation Checklist

Before deploying any model to engineering practice:

☐ **Physical plausibility**: Do predictions make physical sense?  
☐ **Interpolation accuracy**: Low error within training domain?  
☐ **Extrapolation behavior**: Reasonable outside training (if applicable)?  
☐ **Robustness**: Stable under noise and perturbations?  
☐ **Computational cost**: Acceptable for application?  
☐ **Interpretability**: Can you explain predictions to stakeholders?  
☐ **Uncertainty**: Do you know when the model is unreliable?  
☐ **Documentation**: Complete trail for reproducibility?  


---
## Final Thoughts

Machine learning for mechanical systems succeeds when:

**Domain knowledge guides**:
- Feature selection based on physics
- Library construction reflecting expected structure
- Validation against physical constraints

**Data quality is prioritized**:
- Strategic experimental design
- Proper noise characterization
- Adequate coverage of operating conditions

**Methods are combined intelligently**:
- Not "ML vs physics" but "ML with physics"
- Leverage strengths of each approach
- Transparent about limitations

**Engineering judgment prevails**:
- Understanding trumps blind optimization
- Validation over training accuracy
- Simplicity over unnecessary complexity

> "The best model is the simplest one that adequately captures the physics for your specific application."

Thank you for completing this notebook. You now have the tools to:
- Select appropriate ML architectures for mechanical problems
- Train robust models from noisy experimental data
- Validate models systematically
- Make informed engineering decisions

**Next steps**: Apply these methods to your own mechanical systems, always keeping physical plausibility and validation at the forefront.